<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/ons_study_v19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# @title [CELL 1] Setup + Global Variables (V8 Baseline + Bio Proof Nav)
import os, sys, re, csv, json, html, socket, pytz
import pandas as pd
from ftplib import FTP_TLS
from datetime import datetime

try:
    import tqdm
except ImportError:
    os.system('pip install tqdm')
    import tqdm

print("="*60)
print("      [CELL 1] SETUP LOADED (V8 Baseline - Clean Nav)")
print("="*60)

TNG_BASE_URL = "https://yates.one-name.net/tng/verticalchart.php?personID="
TNG_SUFFIX = "&tree=tree1&parentset=0&display=vertical&generations=15"

# Orange background removed from Brick Wall Buster link
NAV_HTML = r"""<style>nav.oldnav ul{display:flex;flex-wrap:wrap;justify-content:center;background-color:#006064!important;border-bottom:2px solid #00acc1!important;margin:0;padding:0;list-style:none} nav.oldnav li{display:inline-block} nav.oldnav a{display:block;padding:10px 15px;text-decoration:none;color:#e0f7fa!important;font-size:14px} nav.oldnav a:hover{background-color:#00838f!important} @media print { nav.oldnav, #nav-slot, .no-print, .action-btn, .control-panel, .tabs { display: none !important; } }</style><nav class="oldnav"><ul><li><a href="/ons-study/research_admin.html" style="color:#ffcc80 !important; font-weight:bold;">Admin Hub</a></li><li><a href="/ons-study/contents.shtml" style="color:#ffcc80 !important; font-weight:bold;">Guide</a></li><li><a href="/ons-study/yates_ancestor_register.shtml">DNA Register</a></li><li><a href="/ons-study/just-trees.shtml">Trees</a></li><li><a href="/ons-study/dna_network.shtml">DNA Network</a></li><li><a href="/ons-study/lineage_proof.html">Lineage Proof</a></li><li><a href="/ons-study/biological_proof.html" style="color:#fff !important; font-weight:bold;">Biological Proof</a></li><li><a href="/ons-study/dna_dossier.html">Forensic Dossier</a></li><li><a href="/ons-study/brick_wall_buster.shtml">Brick Wall Buster</a></li><li><a href="/ons-study/data_glossary.shtml">Data Glossary</a></li><li><a href="https://yates.one-name.net/gengen/images/cousin-calculator.jpg" target="_blank" style="color:#b2dfdb;">Cousin Calc</a></li><li><a href="https://yates.one-name.net/gengen/images/Shared_cM_Project_v4.jpg" target="_blank" style="color:#b2dfdb;">cM Chart</a></li><li><a href="/ons-study/share_dna.shtml" style="background-color:#0277bd; font-weight:bold;">Share DNA</a></li><li><a href="/ons-study/subscribe.shtml" style="background-color:#004d40;">Subscribe</a></li></ul></nav>"""

SITE_INFO = r"""<div class="no-print" style="background:#e0f2f1;border:1px solid #b2dfdb;padding:20px;margin:20px auto;width:90%;border-radius:8px;font-family:sans-serif;"><h3 style="color:#006064;margin-top:0;border-bottom:2px solid #004d40;padding-bottom:10px;">Establishing Kinship Through Collateral DNA Saturation</h3><p style="color:#333;line-height:1.6;font-size:1.05em;margin-bottom:0;"><strong>Methodology:</strong> This register moves beyond the reliance on single "golden matches" to prove kinship. Instead, it employs <em>Collateral DNA Saturation</em>—a method that blends genealogical reasoning with data-driven logic.</p></div>"""

JS_CORE = r"""<script type="text/javascript">(function(){ function textOf(c){var val = c.getAttribute('data-sort') || c.textContent || c.innerText;return (val || '').replace(/\s+/g,' ').trim().toLowerCase();} function sortTable(t,i,d){if(!(t&&t.tBodies&&t.tBodies[0]))return;var tb=t.tBodies[0],r=Array.prototype.slice.call(tb.rows||[]),asc=(d==='asc');r.sort(function(a,b){var A=textOf(a.cells[i]),B=textOf(b.cells[i]),nA=parseFloat(A.replace(/[^0-9.\-]/g,'')),nB=parseFloat(B.replace(/[^0-9.\-]/g,''));if(!isNaN(nA)&&!isNaN(nB))return asc?(nA-nB):(nB-nA);return(A<B)?(asc?-1:1):(A>B)?(asc?1:-1):0;});var f=document.createDocumentFragment();for(var k=0;k<r.length;k++)f.appendChild(r[k]);tb.appendChild(f);} function makeSortable(t){if(!(t&&t.tHead&&t.tHead.rows.length))return;var th=t.tHead.rows[0].cells;for(var i=0;i<th.length;i++){(function(idx){var h=th[idx],d='asc';h.style.cursor='pointer';h.onclick=function(){d=(d==='asc')?'desc':'asc';for(var j=0;j<th.length;j++)th[j].innerHTML=th[j].innerHTML.replace(/\s+\(asc\)|\s+\(desc\)/,'');h.innerHTML+=(d==='asc'?' (asc)':' (desc)');sortTable(t,idx,d);};})(i);}} window.filterTable = function() { var input = document.getElementById("tableSearch"); var filter = input.value.toUpperCase(); var table = document.getElementById("reg-table") || document.querySelector("table.dataframe"); var tr = table.getElementsByTagName("tr"); for (var i = 1; i < tr.length; i++) { var tdArr = tr[i].getElementsByTagName("td"); var found = false; for (var j = 0; j < tdArr.length; j++) { if (tdArr[j]) { var txtValue = tdArr[j].textContent || tdArr[j].innerText; if (txtValue.toUpperCase().indexOf(filter) > -1) { found = true; break; } } } tr[i].style.display = found ? "" : "none"; } } function init(){ var t=document.getElementsByTagName('table'); for(var i=0;i<t.length;i++) if(/\bsortable\b/.test(t[i].className)) makeSortable(t[i]); } if(document.readyState==='loading')document.addEventListener('DOMContentLoaded',init,false);else init(); })();</script>"""

# --- EXACT V8 MAKE_PAGE SIGNATURE RESTORED ---
def make_page(title, content, count, view_type="", extra="", stats_bar=""):
    nav_blk = ""
    if view_type in ['ancestor', 'participant', 'tree_az', 'tree_za', 'proof', 'hot_paths', 'network', 'dossier', 'subscribe', 'share', 'buster', 'singleton']:
        nav_blk = SITE_INFO
    if view_type == 'subscribe' or view_type == 'theory' or view_type == 'share' or view_type == 'glossary':
        nav_blk = ""

    toggle = ""
    print_btn = ""
    search_bar = ""

    if view_type in ['ancestor', 'participant', 'singleton']:
        search_bar = """<div class="no-print" style="margin:20px auto;max-width:600px;text-align:center;"><input type="text" id="tableSearch" onkeyup="filterTable()" placeholder="🔍 Type a name to filter list..." style="width:100%;padding:12px;font-size:16px;border:2px solid #006064;border-radius:4px;"></div>"""

    if view_type in ['ancestor', 'participant', 'singleton']:
        view_name = "Register"
        if view_type == 'singleton': view_name = "Singleton List"
        print_btn = f"""<div class="no-print" style="text-align:center;margin-bottom:15px;"><button onclick="window.print()" style="background:#0277bd;color:white;border:none;padding:10px 20px;border-radius:4px;font-weight:bold;cursor:pointer;font-size:14px;">🖨️ Print {view_name}</button></div>"""

    if view_type == 'ancestor':
        toggle = f"""<div class="no-print" style="text-align:center;padding:10px;margin-bottom:10px;font-family:sans-serif;font-size:14px;background:#e0f7fa;border:1px solid #b2ebf2;"><strong>Sort Register:</strong> &nbsp;<span style="font-weight:bold;color:#006064;">By Ancestral Line</span> &nbsp;|&nbsp; <a href="ons_yates_dna_register_participants.shtml" style="color:#00acc1;text-decoration:none;">By Participant Name</a></div>"""
    elif view_type == 'participant':
        toggle = f"""<div class="no-print" style="text-align:center;padding:10px;margin-bottom:10px;font-family:sans-serif;font-size:14px;background:#e0f7fa;border:1px solid #b2ebf2;"><strong>Sort Register:</strong> &nbsp;<a href="ons_yates_dna_register.shtml" style="color:#00acc1;text-decoration:none;">By Ancestral Line</a> &nbsp;|&nbsp; <span style="font-weight:bold;color:#006064;">By Participant Name</span></div>"""
    elif 'tree' in view_type:
        za = f'<span style="font-weight:bold;color:#000;">Z-A</span>' if 'za' in view_type else f'<a href="just-trees.shtml" style="color:#006064;text-decoration:underline;">Z-A</a>'
        az = f'<span style="font-weight:bold;color:#000;">A-Z</span>' if 'az' in view_type else f'<a href="just-trees-az.shtml" style="color:#006064;text-decoration:underline;">A-Z</a>'
        toggle = f"""<div class="no-print" style="text-align:center;font-family:sans-serif;font-size:16px;margin:15px 0 10px 0;">Individual Yates Family trees: &nbsp; {za} &nbsp;|&nbsp; {az}</div>"""

    return f"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>{title}</title><link rel="stylesheet" href="partials_unified.css"><link rel="stylesheet" href="dna_tree_styles.css">{extra}</head><body id="top"><div class="wrap"><h1 class="centerline">{title}</h1><div id="nav-slot">{stats_bar}{NAV_HTML}</div>{nav_blk}{search_bar}{print_btn}{toggle}{content}</div>{JS_CORE}</body></html>"""

print("✅ Cell 1 Loaded.")

      [CELL 1] SETUP LOADED (V8 Baseline - Clean Nav)
✅ Cell 1 Loaded.


In [24]:
# @title [CELL 3] The Data Engine (V123 - Deep Ancestry Radar)
def run_engine():
    print("="*60)
    print("      [CELL 3] ENGINE STARTING (V123 - DEEP RADAR)...")
    print("="*60)

    import os, sys, re, csv
    from ftplib import FTP_TLS
    from google.colab import userdata
    from datetime import datetime

    CSV_DB = "engine_database.csv"
    if os.path.exists(CSV_DB): os.remove(CSV_DB)

    try:
        HOST = os.environ.get("FTP_HOST") or userdata.get("FTP_HOST")
        USER = os.environ.get("FTP_USER") or userdata.get("FTP_USER")
        PASS = os.environ.get("FTP_PASS") or userdata.get("FTP_PASS")
    except: pass
    REMOTE_SUBDIR = "ons-study"
    KEY_FILE = "match_to_unmasked.csv"
    PROCESSED_GED = "_processed_unmasked.ged"

    def clean_and_standardize(raw_name):
        if not raw_name: return "findme"
        s = raw_name.replace("/", "").strip()
        triggers = ["unknown", "missing", "searching", "still searching", "living", "private", "nee", "nee ?", "wife", "mrs"]
        if s.lower() in triggers or s == "": return "findme"
        if "?" in s: return "findme"
        if "unknown" in s.lower(): return "findme"
        return s

    def get_surname(full_name):
        if not full_name or "findme" in full_name.lower(): return ""
        clean = re.sub(r'\b(jr\.?|sr\.?|iii|iv|esq\.?|m\.d\.?|ph\.d\.?)\b', '', full_name, flags=re.IGNORECASE)
        parts = clean.replace(',', '').split()
        return parts[-1] if parts else ""

    def make_directory_label(name, dates):
        if "findme" in name.lower(): return name
        sur = get_surname(name)
        if not sur: return name
        firsts = re.sub(f"{re.escape(sur)}$", "", name).strip()
        return f"{sur}, {firsts} {dates}"

    print("\n[STEP 1] Resolving Files (Local Priority)...")
    if os.path.exists(KEY_FILE):
        print(f"    ✅ Found {KEY_FILE} locally. Skipping FTP download.")
    else:
        print(f"    🌐 {KEY_FILE} not found locally. Attempting FTP fetch...")
        try:
            ftps = FTP_TLS()
            ftps.connect(HOST, 21); ftps.auth(); ftps.login(USER, PASS); ftps.prot_p()
            try:
                with open(KEY_FILE, "wb") as f: ftps.retrbinary(f"RETR /{REMOTE_SUBDIR}/{KEY_FILE}", f.write)
                print(f"    ✅ Successfully downloaded {KEY_FILE}.")
            except Exception as e:
                print(f"    ⚠️ FTP download failed: {e}")
            ftps.quit()
        except Exception as e:
            print(f"    ⚠️ FTP connection failed: {e}")

    all_files = os.listdir('.')
    ged_files = [f for f in all_files if f.lower().endswith('.ged') and "_processed" not in f.lower()]
    if not ged_files: return print("❌ No GEDCOM found. Please upload one.")
    ged_files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    DEFAULT_GEDCOM = ged_files[0]
    print(f"    👉 Source GEDCOM: {DEFAULT_GEDCOM}")

    def resolve_code(payload):
        m = re.search(r'(\d+)\s*&?\s*([^ \t\n\r\f\v]+)', payload)
        return m.group(2).lower() if m else None

    print("\n[STEP 2] Loading Tester Authority CSV...")
    csv_auth = {}
    if os.path.exists(KEY_FILE):
        with open(KEY_FILE, 'r', errors='replace') as f:
            reader = csv.reader(f)
            for i, row in enumerate(reader):
                if len(row) >= 2:
                    if i == 0 and ("tester" in row[0].lower() or "masked" in row[0].lower() or "code" in row[0].lower()):
                        continue
                    code = row[0].strip().lower()
                    name = row[1].strip()
                    tid = row[2].strip() if len(row) > 2 else ""
                    if tid: tid = "I" + re.sub(r'[^0-9]', '', tid)
                    csv_auth[code] = {"name": name, "id": tid}

    print("\n[STEP 3] Parsing GEDCOM for Study| Tags & Lineages...")
    import shutil
    shutil.copyfile(DEFAULT_GEDCOM, PROCESSED_GED)

    individuals = {}; families = {}; study_testers = {}

    def is_yates(name_str):
        n = (name_str or "").lower()
        return "yates" in n or "yeates" in n or "yate" in n

    current_id = None; current_fam = None; current_tag = None
    with open(PROCESSED_GED, "r", encoding="utf-8", errors="replace") as f:
        for line in f:
            line = line.strip(); parts = line.split(" ", 2)
            if len(parts) < 2: continue
            lvl, tag, val = parts[0], parts[1], parts[2] if len(parts)>2 else ""

            if lvl == "0" and "INDI" in val:
                current_id = tag.replace("@", "")
                individuals[current_id] = {"name": "findme", "famc": None, "fams": [], "match_code": "", "cm": 0, "birt": "", "deat": ""}
                current_fam = None; current_tag = "INDI"
            elif current_id and lvl != "0":
                if tag == "NAME" and lvl == "1":
                    individuals[current_id]["name"] = clean_and_standardize(val)
                elif tag == "FAMC" and lvl == "1":
                    individuals[current_id]["famc"] = val.replace("@", "")
                elif tag == "FAMS" and lvl == "1":
                    individuals[current_id]["fams"].append(val.replace("@", ""))

                elif tag == "NICK" and lvl == "2" and "Study|" in val:
                    tester_code = val.split("Study|")[-1].strip().lower()
                    study_testers[tester_code] = {"id": current_id, "name": individuals[current_id]["name"]}

                elif tag == "NPFX" and lvl == "2":
                    code = resolve_code(val)
                    if code: individuals[current_id]["match_code"] = code.lower()
                    m = re.search(r'^(\d+)|(\d+)\s*cM', val, re.IGNORECASE)
                    if m: individuals[current_id]["cm"] = int(m.group(1) or m.group(2))

                elif tag == "BIRT": current_tag = "BIRT"
                elif tag == "DEAT": current_tag = "DEAT"
                elif tag == "DATE" and current_tag:
                    m = re.search(r'\d{4}', val)
                    if m: individuals[current_id][current_tag.lower()] = m.group(0)
                    current_tag = None

            if lvl == "0" and "FAM" in val:
                current_fam = tag.replace("@", "")
                families[current_fam] = {"husb": None, "wife": None}
                current_id = None
            elif current_fam and lvl != "0":
                if tag == "HUSB": families[current_fam]["husb"] = val.replace("@", "")
                elif tag == "WIFE": families[current_fam]["wife"] = val.replace("@", "")

    def get_parents(pid):
        if not pid or pid not in individuals: return None, None
        famc = individuals[pid]["famc"]
        if not famc or famc not in families: return None, None
        return families[famc]["husb"], families[famc]["wife"]

    def get_mother_surname(pid):
        if not pid: return ""
        _, mom_id = get_parents(pid)
        if mom_id and mom_id in individuals: return get_surname(individuals[mom_id]["name"])
        return ""

    def to_spanish_name(pid, current_name):
        if "findme" in current_name.lower(): return current_name
        mom_surname = get_mother_surname(pid)
        if not mom_surname or "findme" in mom_surname.lower(): return current_name
        if mom_surname.lower() not in current_name.lower(): return f"{current_name}-{mom_surname}"
        return current_name

    # 🌟 NEW: THE DEEP ANCESTRY RADAR
    # This recursively checks if a specific person has ANY Yates in their documented ancestry
    yates_memo = {}
    def has_yates_ancestry(pid):
        if not pid or pid not in individuals: return False
        if pid in yates_memo: return yates_memo[pid]

        # Base case: Is this person a Yates?
        if is_yates(individuals[pid].get("name", "")):
            yates_memo[pid] = True
            return True

        # Recursive case: Check their parents
        dad_id, mom_id = get_parents(pid)
        res = has_yates_ancestry(dad_id) or has_yates_ancestry(mom_id)
        yates_memo[pid] = res
        return res

    def climb_full_line(start_id):
        curr = start_id; lineage_data = []
        while curr:
            p = individuals.get(curr)
            if not p: break
            spanish_name = to_spanish_name(curr, p["name"])
            spouse_name = "findme"; spouse_id = None
            if p["fams"]:
                fid = p["fams"][0]
                if fid in families:
                    f = families[fid]
                    sid = f["wife"] if f["husb"] == curr else f["husb"]
                    if sid and sid in individuals:
                        spouse_name = individuals[sid]["name"]; spouse_id = sid
            spouse_spanish = to_spanish_name(spouse_id, spouse_name) if spouse_id else spouse_name
            lineage_data.append({"name": spanish_name, "raw_name": p["name"], "id": curr, "spouse": spouse_spanish, "spouse_raw": spouse_name, "spouse_id": spouse_id})

            dad_id, mom_id = get_parents(curr)
            if not dad_id and not mom_id: break

            # 🌟 USE THE RADAR TO STEER THE CLIMB
            dad_has_yates = has_yates_ancestry(dad_id)
            mom_has_yates = has_yates_ancestry(mom_id)

            if dad_has_yates and not mom_has_yates: curr = dad_id
            elif mom_has_yates and not dad_has_yates: curr = mom_id
            else: curr = dad_id if dad_id else mom_id

        return lineage_data

    def format_dates(uid):
        if not uid or uid not in individuals: return "findme"
        b = individuals[uid]["birt"] or "findme"
        d = individuals[uid]["deat"] or "findme"
        b = re.sub(r'\?', 'findme', b); d = re.sub(r'\?', 'findme', d)
        if b == "findme" and d == "findme": return "findme"
        return f"({b} - {d})"

    testers = {}
    for code, data in csv_auth.items():
        testers[code] = {"name": data["name"], "id": data["id"]}

    for code, data in study_testers.items():
        if code not in testers:
            testers[code] = {"name": data["name"], "id": data["id"]}
        elif not testers[code]["id"]:
            testers[code]["id"] = data["id"]

    for kcode, tdata in testers.items():
        t_lin = ""; t_pids = ""
        if tdata["id"] and tdata["id"] in individuals:
            lin_data = climb_full_line(tdata["id"])
            if lin_data:
                full = list(reversed(lin_data))
                t_lin = " -> ".join([x["name"] for x in full])
                t_pids = ",".join([x["id"] for x in full])
        tdata["lineage_str"] = t_lin
        tdata["path_ids"] = t_pids

    print("\n[STEP 4] Constructing Database...")
    rows = []
    for uid, p in individuals.items():
        if p["match_code"]: # It's a found match!
            kit_code = p["match_code"]

            if kit_code in testers:
                t_name = testers[kit_code]["name"]
                t_id = testers[kit_code]["id"]
                t_lin = testers[kit_code]["lineage_str"]
                t_pids = testers[kit_code]["path_ids"]
                tester_display = f"{t_name} [{t_id}]" if t_id else f"{t_name} [{kit_code}]"
            else:
                t_name = kit_code
                t_id = ""
                t_lin = ""
                t_pids = ""
                tester_display = f"{kit_code} [{kit_code}]"

            lineage_data = climb_full_line(uid)
            if not lineage_data: continue

            full_line = list(reversed(lineage_data))
            gen1 = full_line[0]

            top_name = gen1["raw_name"]
            top_dates = format_dates(gen1["id"])
            spouse_name = gen1["spouse_raw"]
            spouse_id = gen1["spouse_id"]
            spouse_dates = format_dates(spouse_id)

            if spouse_name != "findme":
                husb_sur = get_surname(top_name); wife_sur = get_surname(spouse_name)
                if husb_sur.lower() == wife_sur.lower(): spouse_name += f" (née {wife_sur})"

            pair_dated = f"{top_name} {top_dates}"
            if spouse_name != "findme": dir_label = make_directory_label(top_name, top_dates) + f" & {spouse_name}"
            else: dir_label = make_directory_label(top_name, top_dates)

            if spouse_name != "findme": pair_dated += f" & {spouse_name} {spouse_dates}"
            pair_simple = f"{top_name} & {spouse_name}" if spouse_name != "findme" else top_name

            clean_top = re.sub(r'[^a-zA-Z0-9]', '', top_name)
            clean_sp = re.sub(r'[^a-zA-Z0-9]', '', spouse_name.split('(')[0]) if spouse_name != "findme" else "ZZZ"
            sort_key = f"{clean_top}_{clean_sp}"

            path_names = []
            for i, x in enumerate(full_line):
                if i == 0: path_names.append(pair_dated)
                else: path_names.append(x["name"])

            lineage_str = " -> ".join(path_names)
            path_ids = ",".join([x["id"] for x in full_line])

            _, fa1_mom_id = get_parents(gen1["id"])
            fa1_mother = to_spanish_name(fa1_mom_id, individuals[fa1_mom_id]["name"]) if fa1_mom_id else "findme"

            fa2_mother = "findme"
            if spouse_id:
                _, fa2_mom_id = get_parents(spouse_id)
                if fa2_mom_id: fa2_mother = to_spanish_name(fa2_mom_id, individuals[fa2_mom_id]["name"])

            rows.append({
                "Tester_Code": kit_code,
                "Tester_Name": t_name,
                "Tester_ID": t_id,
                "Tester_Display": tester_display,
                "Tester_Lineage": t_lin,
                "Tester_Path_IDs": t_pids,
                "Match_Name": p["name"],
                "Match_ID": uid,
                "cM": p["cm"],
                "Match_Lineage": lineage_str,
                "Match_Path_IDs": path_ids,
                "Authority_Directory_Label": dir_label,
                "Authority_FirstAncestor": pair_simple,
                "Authority_FirstAncestor_alpha": sort_key,
                "Authority_FirstAncestor_dated": pair_dated,
                "fa_1 extracted": top_name, "fa_1_Dates": top_dates, "fa_1_Mother": fa1_mother,
                "fa_2 extracted": spouse_name, "fa_2 Dates": spouse_dates, "fa_2_Mother": fa2_mother,
                "Gen_Count": len(full_line)
            })

    rows.sort(key=lambda r: r["Authority_Directory_Label"])

    fieldnames = [
        "Tester_Code", "Tester_Name", "Tester_ID", "Tester_Display",
        "Tester_Lineage", "Tester_Path_IDs",
        "Match_Name", "Match_ID", "cM", "Match_Lineage", "Match_Path_IDs",
        "Authority_Directory_Label", "Authority_FirstAncestor", "Authority_FirstAncestor_alpha", "Authority_FirstAncestor_dated",
        "fa_1 extracted", "fa_1_Dates", "fa_1_Mother",
        "fa_2 extracted", "fa_2 Dates", "fa_2_Mother",
        "Gen_Count"
    ]

    with open(CSV_DB, "w", encoding="iso-8859-15", newline="", errors="replace") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
        writer.writeheader(); writer.writerows(rows)

    print(f"\n[SUCCESS] Engine V123 Complete. Saved {len(rows)} verified matches to {CSV_DB}.")

print("✅ Cell 3 (Engine V123 - Deep Ancestry Radar) Loaded.")

✅ Cell 3 (Engine V123 - Deep Ancestry Radar) Loaded.


In [28]:
# @title [CELL 4] The Template Library (Pristine UI + ALL CSS Vars Patched)
print("="*60)
print("      [CELL 4] TEMPLATE LIBRARY LOADING (Pristine UI Restored)...")
print("="*60)

# 🌟 0. IMPENETRABLE CATCH-ALL CSS VARIABLES
# (This prevents any "not defined" errors from the Publisher)
CONTENTS_CSS = ""
SHARE_CSS = ""
GLOSS_CSS = ""
GLOSSARY_CSS = ""
SUBSCRIBE_CSS = ""
THEORY_CSS = ""
TREE_CSS = ""
ADMIN_CSS = ""
BUST_CSS = ""
DOSS_CSS = ""
PROOF_CSS = ""
BIO_CSS = ""
REGISTER_CSS = "<style>.table-scroll-wrapper{overflow-x:auto; background:white; padding:20px; border-radius:8px; box-shadow:0 4px 6px rgba(0,0,0,0.1);}</style>"

# 🌟 1. GLOBAL CSS, PRISTINE NAV & CORE SCRIPTS
NAV_HTML = r"""<style>nav.oldnav ul{display:flex;flex-wrap:wrap;justify-content:center;background-color:#006064!important;border-bottom:2px solid #00acc1!important;margin:0;padding:0;list-style:none} nav.oldnav li{display:inline-block} nav.oldnav a{display:block;padding:10px 15px;text-decoration:none;color:#e0f7fa!important;font-size:14px} nav.oldnav a:hover{background-color:#00838f!important} @media print { nav.oldnav, #nav-slot, .no-print { display: none !important; } }</style><nav class="oldnav"><ul><li><a href="/ons-study/research_admin.html" style="color:#ffcc80 !important; font-weight:bold;">Admin Hub</a></li><li><a href="/ons-study/contents.shtml" style="color:#ffcc80 !important; font-weight:bold;">Guide</a></li><li><a href="/ons-study/ons_yates_dna_register.shtml">DNA Register</a></li><li><a href="/ons-study/just-trees.shtml">Trees</a></li><li><a href="/ons-study/dna_network.shtml">DNA Network</a></li><li><a href="/ons-study/lineage_proof.html">Lineage Proof</a></li><li><a href="/ons-study/biological_proof.html">Biological Proof</a></li><li><a href="/ons-study/proof_consolidator.html" style="background-color:#4a148c; color:#fff !important; font-weight:bold; border-left:1px solid #7c43bd; border-right:1px solid #7c43bd;">Consolidator</a></li><li><a href="/ons-study/dna_dossier.html">Forensic Dossier</a></li><li><a href="/ons-study/brick_wall_buster.shtml">Brick Wall Buster</a></li><li><a href="/ons-study/data_glossary.shtml">Data Glossary</a></li><li><a href="https://yates.one-name.net/gengen/images/cousin-calculator.jpg" target="_blank" style="color:#b2dfdb;">Cousin Calc</a></li><li><a href="/ons-study/share_dna.shtml" style="background-color:#0277bd; font-weight:bold;">Share DNA</a></li><li><a href="/ons-study/subscribe.shtml" style="background-color:#004d40; font-weight:bold;">Subscribe</a></li></ul></nav>"""
SITE_INFO = r"""<div class="no-print" style="background:#e0f2f1;border:1px solid #b2dfdb;padding:20px;margin:20px auto;width:90%;border-radius:8px;font-family:sans-serif;"><h3 style="color:#006064;margin-top:0;border-bottom:2px solid #004d40;padding-bottom:10px;">Establishing Kinship Through Collateral DNA Saturation</h3><p style="color:#333;line-height:1.6;margin-bottom:0;">This register employs <em>Collateral DNA Saturation</em>—a method blending genealogical reasoning with data-driven logic to prove connections using multiple independent DNA cousins.</p></div>"""
CSS_BASE = r"""body{font-family:'Segoe UI',sans-serif;background:#f0f2f5;padding:20px;display:flex;flex-direction:column;min-height:100vh;margin:0;} .wrap{flex:1;} .proof-card{background:white;max-width:1100px;margin:20px auto;border-radius:8px;box-shadow:0 4px 15px rgba(0,0,0,0.1);padding:40px} .badge{padding:5px 10px;border-radius:4px;font-weight:bold;font-size:0.85em;text-transform:uppercase;border:1px solid #ccc;} .badge-platinum{background:#eceff1;color:#263238} .badge-gold{background:#fff8e1;color:#f57f17} .badge-silver{background:#f5f5f5;color:#616161} .badge-bronze{background:#efebe9;color:#5d4037} table{width:100%;border-collapse:collapse;margin-top:15px;margin-bottom:40px;font-family:'Georgia',serif;font-size:15px;} th{background:#eceff1;color:#263238;padding:12px;text-align:left;border-bottom:2px solid #000;} td{padding:12px;border-bottom:1px solid #ddd;vertical-align:top;} @media print{ .no-print{display:none !important;} .only-print{display:block !important;} .proof-card{box-shadow:none;border:none;padding:0;margin:0;} body{background:white;padding:0;display:block;} th{background:#f0f0f0 !important;color:#000 !important;} .badge{border:1px solid #000;color:#000;background:transparent !important;} .legal-footer{background:transparent !important; border-top:2px solid #000 !important; color:#000 !important; page-break-inside:avoid !important; padding:10px 0 !important; margin-top:30px !important;} } .only-print{display:none;}"""
LEGAL_FOOTER_TMPL = r"""<div class="legal-footer no-print" style="margin-top:50px;padding:20px;background:#f4f4f4;border-top:1px solid #ddd;text-align:center;color:#666;font-family:sans-serif;font-size:0.85em;clear:both;"><p style="margin-bottom:5px;font-size:1.1em;color:#333;"><strong>&copy; __YEAR__ Ronald Eugene Yates. All Rights Reserved.</strong></p><p style="margin-bottom:5px;">Generated by <em>The Forensic Genealogy Publisher&trade;</em></p><p style="font-style:italic;color:#888;margin-bottom:0;max-width:800px;margin-left:auto;margin-right:auto;">The terms "Forensic Handshake", "Brick Wall Buster", and "Collateral Saturation" are trademarks of the Yates One-Name Study.</p></div>"""
BTT_BTN = r"""<style>.btt{position:fixed;bottom:20px;right:20px;background:#00838f;color:white;padding:10px 15px;text-decoration:none;border-radius:4px;font-weight:bold;box-shadow:0 2px 5px rgba(0,0,0,0.3);z-index:1000;opacity:0.9;} .btt:hover{opacity:1;background:#006064;} @media print { .btt { display: none !important; } }</style><a href="#top" class="btt no-print">⬆️ Top</a>"""
JS_CORE = r"""<script type="text/javascript">(function(){ function textOf(c){var val = c.getAttribute('data-sort') || c.textContent || c.innerText;return (val || '').replace(/ +/g,' ').trim().toLowerCase();} function sortTable(t,i,d){if(!(t&&t.tBodies&&t.tBodies[0]))return;var tb=t.tBodies[0],r=Array.prototype.slice.call(tb.rows||[]),asc=(d==='asc');r.sort(function(a,b){var A=textOf(a.cells[i]),B=textOf(b.cells[i]),nA=parseFloat(A.replace(/[^0-9.-]/g,'')),nB=parseFloat(B.replace(/[^0-9.-]/g,''));if(!isNaN(nA)&&!isNaN(nB))return asc?(nA-nB):(nB-nA);return(A<B)?(asc?-1:1):(A>B)?(asc?1:-1):0;});var f=document.createDocumentFragment();for(var k=0;k<r.length;k++)f.appendChild(r[k]);tb.appendChild(f);} function makeSortable(t){if(!(t&&t.tHead&&t.tHead.rows.length))return;var th=t.tHead.rows[0].cells;for(var i=0;i<th.length;i++){(function(idx){var h=th[idx],d='asc';h.style.cursor='pointer';h.onclick=function(){d=(d==='asc')?'desc':'asc';for(var j=0;j<th.length;j++)th[j].innerHTML=th[j].innerHTML.replace(' (asc)','').replace(' (desc)','');h.innerHTML+=(d==='asc'?' (asc)':' (desc)');sortTable(t,idx,d);};})(i);}} window.filterTable = function() { var input = document.getElementById("tableSearch"); var filter = input.value.toUpperCase(); var table = document.getElementById("reg-table") || document.querySelector("table.dataframe"); var tr = table.getElementsByTagName("tr"); for (var i = 1; i < tr.length; i++) { var tdArr = tr[i].getElementsByTagName("td"); var found = false; for (var j = 0; j < tdArr.length; j++) { if (tdArr[j]) { var txtValue = tdArr[j].textContent || tdArr[j].innerText; if (txtValue.toUpperCase().indexOf(filter) > -1) { found = true; break; } } } tr[i].style.display = found ? "" : "none"; } } function init(){ var t=document.getElementsByTagName('table'); for(var i=0;i<t.length;i++) if(t[i].className.indexOf('sortable') !== -1) makeSortable(t[i]); } if(document.readyState==='loading')document.addEventListener('DOMContentLoaded',init,false);else init(); })();</script>"""

# 🌟 2. PRISTINE STATIC PAGES (JSON EXTRACTION)
CONTENTS_CONTENT = r"""<style>.guide-grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(300px,1fr));gap:25px;max-width:1200px;margin:30px auto;font-family:sans-serif;}.guide-card{background:white;padding:25px;border-radius:8px;border-left:5px solid #006064;box-shadow:0 4px 10px rgba(0,0,0,0.05);transition:transform 0.2s;text-align:left;}.guide-card:hover{transform:translateY(-5px)}.card-title{font-size:1.4em;font-weight:bold;color:#004d40;margin-top:0}.card-why{color:#b71c1c;font-weight:bold;margin:10px 0 5px 0;font-size:0.9em;text-transform:uppercase}.card-what{color:#555;font-size:1em;line-height:1.5;margin-bottom:20px}.card-btn{display:inline-block;padding:10px 20px;background:#00838f;color:white;text-decoration:none;border-radius:4px;font-weight:bold}.card-btn:hover{background:#006064}</style><div style="text-align:center;max-width:800px;margin:0 auto 20px auto;color:#444;font-size:1.1em;font-family:sans-serif;">This site transforms raw DNA data into forensic genealogical evidence.</div><div class="guide-grid"><div class="guide-card"><h2 class="card-title">1. The DNA Register</h2><div class="card-why">Why View This?</div><div class="card-what">To see the raw evidence. This is the master list of all matches in the study, sorted by ancestral line.</div><a href="ons_yates_dna_register.shtml" class="card-btn">Open Register</a></div><div class="guide-card"><h2 class="card-title">2. DNA Network</h2><div class="card-why">Why View This?</div><div class="card-what">To see the big picture. Visual clusters showing which ancestral lines are genetically proven by multiple testers.</div><a href="dna_network.shtml" class="card-btn">View Network</a></div><div class="guide-card"><h2 class="card-title">3. Lineage Proof Engine</h2><div class="card-why">Why View This?</div><div class="card-what">To verify a connection. An interactive tool that tests if a specific ancestor is biologically confirmed.</div><a href="lineage_proof.html" class="card-btn">Run Proof</a></div><div class="guide-card" style="border-left-color:#f57f17;"><h2 class="card-title" style="color:#e65100;">4. Brick Wall Buster</h2><div class="card-why" style="color:#bf360c;">Why View This?</div><div class="card-what">Break through a dead end. Suggests which family line you likely belong to based on match dominance.</div><a href="brick_wall_buster.shtml" class="card-btn" style="background:#ef6c00;">Bust This Wall</a></div><div class="guide-card"><h2 class="card-title">5. Forensic Dossier</h2><div class="card-why">Why View This?</div><div class="card-what">Get your "Scorecard." Generate forensic reports on yourself or an ancestor, grading evidence strength.</div><a href="dna_dossier.html" class="card-btn">Create Dossier</a></div><div class="guide-card"><h2 class="card-title">6. Research Admin Hub</h2><div class="card-why">Why View This?</div><div class="card-what">For study managers. High-level audit showing participant statistics, masked IDs, and study metrics.</div><a href="research_admin.html" class="card-btn" style="background:#455a64;">Admin Access</a></div><div class="guide-card"><h2 class="card-title">7. Data Glossary</h2><div class="card-why">Why View This?</div><div class="card-what">Understand the terms. Definitions for forensic terms like "Keystone Tester" and "Spanish naming."</div><a href="data_glossary.shtml" class="card-btn" style="background:#78909c;">Read Glossary</a></div><div class="guide-card" style="border-left-color:#fbc02d;"><h2 class="card-title">8. Corrections</h2><div class="card-why">See Something?</div><div class="card-what">Genealogy is collaboration. If you can solve a mystery, tell us. Include the <strong>Person ID (e.g. I1234)</strong>.</div><a href="mailto:ron@yates.one-name.net" class="card-btn" style="background:#f9a825;color:#333;">Email Correction</a></div></div>"""
SHARE_CONTENT = r"""<div style="max-width:900px; margin:0 auto; font-family:sans-serif; color:#333;"><div style="background:white; padding:40px; border-radius:8px; box-shadow:0 2px 10px rgba(0,0,0,0.1);"><h2 style="color:#006064; margin-top:0; border-bottom:2px solid #00838f; padding-bottom:10px;">Share Your Ancestry DNA Matches</h2><p style="font-size:1.1em; line-height:1.6;">Ancestry provides a built-in sharing feature that allows you to grant limited access to your DNA matches <strong>without sharing your personal account details</strong>. You remain in full control of your account at all times.</p><h3 style="color:#00838f; margin-top:30px;">How sharing works on Ancestry</h3><p>From your AncestryDNA Settings page, you may invite another individual by email and assign them the <strong>Collaborator</strong> role. This allows the study administrator to analyze your match list to build Forensic Handshakes.</p><div style="background:#f1f8e9; padding:25px; border-radius:8px; border:1px solid #c5e1a5; margin-top:30px;"><h3 style="color:#33691e; margin-top:0;">How to share for the Yates One-Name Study</h3><ol style="line-height:1.8; font-size:1.1em; color:#333;"><li>Open your <strong>AncestryDNA Settings</strong>.</li><li>Use the <strong>Invite</strong> function.</li><li>Send the invitation to <strong>Ron Yates</strong> at: <br><span style="font-family:monospace; font-weight:bold; background:white; padding:5px 10px; border:1px solid #aaa; border-radius:3px; display:inline-block; margin-top:5px; color:#c62828;">yatesvilleron@gmail.com</span></li><li>Assign the role <strong>Collaborator</strong>.</li></ol><p style="font-size:0.95em; color:#558b2f; margin-top:15px; background:white; padding:10px; border-radius:4px; border:1px solid #dcedc8;"><strong>Why Collaborator?</strong> The <em>Collaborator</em> role allows Ron to review shared matches to perform triangulation logic.</p></div></div></div>"""
GLOSSARY_CONTENT = r"""<div style="max-width:900px; margin:0 auto; font-family:sans-serif; color:#333;"><div style="background:white; padding:40px; border-radius:8px; box-shadow:0 2px 10px rgba(0,0,0,0.1);"><h2 style="color:#006064; margin-top:0; border-bottom:2px solid #00838f; padding-bottom:10px;">Study Terminology & Methods</h2><div style="margin-bottom:30px; border-left:5px solid #0277bd; padding-left:20px;"><h3 style="color:#01579b; margin-top:0;">Forensic Handshake</h3><p style="line-height:1.6;">A robust verification method used to confirm ancestral connections. Unlike simple shared matches, a Forensic Handshake requires triangulation between three or more independent DNA kits that descend from different branches of a target ancestor. This cross-verification ensures that the genetic link is genuine and not a result of chance or endogamy.</p></div><div style="margin-bottom:30px; border-left:5px solid #ef6c00; padding-left:20px;"><h3 style="color:#e65100; margin-top:0;">Yates Bio Partners</h3><p style="line-height:1.6;">A collaborative designation for researchers who actively contribute DNA data to the study.</p><p style="line-height:1.6; background:#fff3e0; padding:15px; border-radius:4px;"><strong>Naming Convention:</strong> To accurately track lineages across generations, Bio Partners adopt the <em>Spanish Naming Method</em> for their study profiles.<br><br><strong>Format:</strong> <code>Forename Paternal-Maternal</code><br><strong>Example:</strong> <em>Robert Yates-Smith</em> indicates Robert's father was a Yates and his mother was a Smith.</p></div></div></div>"""
SUBSCRIBE_CONTENT = r"""<div style="background:white; padding:40px; border-radius:8px; box-shadow:0 4px 15px rgba(0,0,0,0.1); max-width:800px; margin:20px auto; text-align:center; font-family:sans-serif;"><h2 style="color:#006064; margin-top:0;">Join the Yates Research Community</h2><div style="background:#e0f2f1; padding:25px; border-radius:8px; border:1px solid #b2dfdb; display:inline-block;"><h3 style="margin-top:0; color:#004d40;">📧 One-Click Subscribe</h3><p>Participation in the Yates DNA Study is completely free.</p><a href="mailto:yates-one-name-study+subscribe@groups.io?subject=Subscribe" style="display:inline-block; padding:15px 30px; background:#00838f; color:white; text-decoration:none; border-radius:5px; font-weight:bold; margin-top:10px;">Subscribe Now</a></div></div>"""
THEORY_CONTENT = r"""<div class="proof-card"><h2 style="color:#006064; border-bottom:2px solid #00acc1; padding-bottom:10px;">The Yates DNA Strategy: Collateral Saturation</h2><p>Traditional genetic genealogy often relies on single "golden matches" to prove a connection. The Yates DNA Strategy rejects this due to the inherent noise in autosomal DNA (e.g., pedigree collapse, false segments).</p><p>Instead, we use <strong>Collateral Saturation</strong>. We require multiple, independent descendants from distinct branches of an ancestor to all share DNA with each other. When these independent lines converge mathematically, it creates a "Forensic Handshake" that proves the lineage beyond a reasonable doubt.</p></div>"""

# 🌟 3. PROOF CONSOLIDATOR
CONSOLIDATOR_CSS = r"""<style>.consol-panel { background: #f3e5f5; border: 1px solid #ab47bc; padding: 25px; border-radius: 8px; margin-bottom: 25px; font-family: 'Segoe UI', sans-serif; text-align: center; } .consol-panel select { padding: 8px; font-size: 14px; width: 100%; border: 1px solid #7b1fa2; border-radius: 4px; } .consol-btn { background: #4a148c; color: white; border: none; padding: 12px 25px; font-size: 16px; font-weight: bold; border-radius: 4px; cursor: pointer; box-shadow: 0 4px 6px rgba(0,0,0,0.1); margin-top: 10px; } .consol-btn:hover { background: #38006b; } .academic-brief { background: white; max-width: 1000px; margin: 0 auto 30px auto; padding: 60px 80px; box-shadow: 0 4px 20px rgba(0,0,0,0.1); font-family: 'Georgia', serif; color: #000; line-height: 1.6; } .title-page { page-break-after: always; display: flex; flex-direction: column; justify-content: center; min-height: 70vh; padding: 20px; } .brief-header { text-align: center; border-bottom: 3px solid #000; padding-bottom: 20px; margin-bottom: 30px; } .brief-header h1 { font-size: 28px; text-transform: uppercase; margin: 0; letter-spacing: 1px; color: #000; } .brief-header p { font-size: 16px; font-style: italic; color: #444; margin: 5px 0 0 0; } .brief-meta { background: #fafafa; border: 1px solid #ddd; padding: 20px; margin-bottom: 30px; font-size: 15px; } .brief-meta strong { color: #000; } .brief-section-title { font-size: 18px; text-transform: uppercase; border-bottom: 1px solid #ccc; padding-bottom: 5px; margin-top: 40px; margin-bottom: 20px; font-weight: bold; } .collateral-list { margin-left: 20px; font-size: 15px; } .collateral-list li { margin-bottom: 8px; } .brief-table { width: 100%; border-collapse: collapse; margin-top: 15px; font-size: 14px; } .brief-table th { background: #f0f0f0; color: #000; border-bottom: 2px solid #000; border-top: 1px solid #000; padding: 10px; text-align: left; } .brief-table td { padding: 10px; border-bottom: 1px solid #ddd; vertical-align: middle; } .brief-table tr { page-break-inside: avoid; } .verdict-stamp { border: 3px double #4a148c; padding: 20px; text-align: center; margin-top: 40px; font-size: 18px; font-weight: bold; color: #4a148c; background: #fafafa; text-transform: uppercase; letter-spacing: 1px; } @media print { .no-print { display: none !important; } body { background: white; padding: 0; } .academic-brief { box-shadow: none; padding: 0; max-width: 100%; border: none; margin-bottom: 0; } .verdict-stamp { border: 3px double #000; color: #000; } }</style>"""
APPENDIX_A_HTML = r"""<div class="academic-brief" style="max-width: 1000px; margin-top: 40px; page-break-before: always; text-align: left; padding: 60px 80px;"><h2 style="color: #4a148c; border-bottom: 2px solid #ccc; padding-bottom: 5px; margin-top:0; font-size:22px; text-transform:uppercase;">Appendix A: CSS Field Definitions &amp; Data Sources</h2><p><b>Purpose:</b> This appendix defines each field used in the Collateral Saturation Score (CSS) matrix...</p></div>"""
CONSOLIDATOR_JS = r"""<script>__JS_GLOBALS__
function cleanNum(str){let res="";for(let i=0;i<str.length;i++)if(str[i]>='0'&&str[i]<='9')res+=str[i];return res;}
const partSel = document.getElementById('testerSelect'); const ancSel = document.getElementById('ancestorSelect');
const validTesters = DB.filter(r => r.t_names && r.t_names.trim() !== "");
const uniqueTesters = [...new Set(validTesters.map(r => r.participant))];
uniqueTesters.sort((a, b) => { const keyA = DATA.participants[a] ? DATA.participants[a].sort_key : a.toLowerCase(); const keyB = DATA.participants[b] ? DATA.participants[b].sort_key : b.toLowerCase(); if (keyA === keyB) return a.localeCompare(b); return keyA.localeCompare(keyB); });
if(partSel) { uniqueTesters.forEach(t => { const o = document.createElement('option'); o.value = t; o.innerText = t; partSel.appendChild(o); }); }
const allAncestors = Object.keys(DATA.ancestors).sort((a,b) => DATA.ancestors[a].name.localeCompare(DATA.ancestors[b].name));
if(ancSel) { allAncestors.forEach(k => { const o = document.createElement('option'); o.value = DATA.ancestors[k].name; o.innerText = DATA.ancestors[k].name; ancSel.appendChild(o); }); }
function getCSS(testerArray, customName = null) {
    const isGroup = testerArray.length > 1; const pName = isGroup ? (customName || `VIRTUAL GROUP (${testerArray.length} Kits)`) : testerArray[0];
    const myMatches = DB.filter(m => testerArray.includes(m.participant) && m.ancestor !== 'No Matches' && m.ancestor);
    let PM = myMatches.length; if(PM === 0) return null;
    let dirs = {}; myMatches.forEach(m => { dirs[m.ancestor] = (dirs[m.ancestor] || 0) + 1; });
    let sortedDirs = Object.entries(dirs).sort((a,b) => b[1] - a[1]);
    let HC_T = sortedDirs.length > 0 ? sortedDirs[0][1] : PM; let HC_2 = sortedDirs.length > 1 ? sortedDirs[1][1] : 0;
    let TB = 0; let NS = 0; let BR = 0; let idCounts = {};
    myMatches.forEach(m => { if(m.search_ids) { let ids = m.search_ids.split(',').map(x=>cleanNum(x)); ids.forEach(id => { if(id) idCounts[id] = (idCounts[id] || 0) + 1; }); } });
    let highestHeat = 0; let targetID = null;
    for (let [id, count] of Object.entries(idCounts)) { const nodeMatches = DB.filter(m => m.search_ids && m.search_ids.split(',').map(x=>cleanNum(x)).includes(id)); const nodeUniqueKits = new Set(nodeMatches.map(m => m.participant)).size; if (nodeUniqueKits > highestHeat) { highestHeat = nodeUniqueKits; targetID = id; } }
    NS = highestHeat;
    if (targetID) { const collaterals = DB.filter(m => m.search_ids && m.search_ids.split(',').map(x=>cleanNum(x)).includes(targetID)); TB = new Set(collaterals.map(m => m.participant)).size; let branches = new Set(); collaterals.forEach(r => { const ids = r.search_ids.split(',').map(x => cleanNum(x)); const names = r.search_names.split('|'); let idx = ids.indexOf(targetID); if(idx !== -1 && idx + 1 < names.length) { branches.add(names[idx+1].replace(/findme/gi, '?').split(' (')[0].trim()); } else { branches.add("Direct Descendant"); } }); BR = branches.size; }
    let DR = HC_T / (HC_2 > 0 ? HC_2 : 1); const norm = (val, cap) => Math.min(1, Math.log(1+val) / Math.log(1+cap));
    let PM_n = norm(PM, 150); let HC_n = norm(HC_T, 100); let DR_n = norm(DR, 10); let TB_n = norm(TB, 40); let NS_n = norm(NS, 150); let BR_n = 0;
    if(BR >= 6) BR_n = 1.0; else if(BR === 5) BR_n = 0.85; else if(BR === 4) BR_n = 0.70; else if(BR === 3) BR_n = 0.50; else if(BR === 2) BR_n = 0.25;
    let ST_str = "FAIL"; let ST_val = 0.60;
    if (PM >= 15) { if(BR >= 3 && DR >= 1.5) { ST_str = "PASS"; ST_val = 1.0; } else if(BR >= 2) { ST_str = "PARTIAL"; ST_val = 0.85; } }
    let weightedSum = (PM_n * 1.0) + (HC_n * 1.0) + (DR_n * 1.5) + (TB_n * 1.0) + (BR_n * 2.0) + (NS_n * 1.0); let cssBase = 100 * (weightedSum / 7.5); let cssFinal = cssBase * ST_val;
    return { pName, PM, HC_T, HC_2, DR, TB, BR, NS, ST_str, cssFinal, isGroup };
}
function getMatrixHTML(vgCSS = null) {
    let matrixRows = []; uniqueTesters.forEach(t => { let c = getCSS([t]); if(c) matrixRows.push(c); }); if (vgCSS && vgCSS.isGroup) matrixRows.push(vgCSS); matrixRows.sort((a,b) => b.cssFinal - a.cssFinal);
    let tableHTML = `<table class="brief-table sortable" style="text-align:center; font-family:sans-serif;"><thead><tr><th style="text-align:left; cursor:pointer; width:22%;">Participant Kit</th><th title="Proper Matches" style="cursor:pointer;">PM</th><th title="Target Handshakes" style="cursor:pointer;">HC-T</th><th title="Secondary Handshakes" style="cursor:pointer;">HC-2</th><th title="Dominance Ratio" style="cursor:pointer;">DR</th><th title="Unique Testers" style="cursor:pointer;">TB</th><th title="Independent Branches" style="cursor:pointer;">BR</th><th title="Node Saturation" style="cursor:pointer;">NS</th><th title="Stability" style="cursor:pointer;">ST</th><th style="background:#f3e5f5; color:#4a148c; cursor:pointer;">CSS v2a</th></tr></thead><tbody>`;
    matrixRows.forEach(r => { let stColor = r.ST_str === "PASS" ? "green" : (r.ST_str === "PARTIAL" ? "#f57f17" : "red"); let brStr = r.BR >= 6 ? "&ge;6" : r.BR; let rowStyle = r.isGroup ? 'background:#fff8e1; border:2px solid #fbc02d;' : ''; let nameFmt = r.isGroup ? `<span style="color:#f57f17; font-weight:bold;">★ ${r.pName}</span>` : `<strong>${r.pName}</strong>`; tableHTML += `<tr style="${rowStyle}"><td style="text-align:left;" data-sort="${r.pName}">${nameFmt}</td><td>${r.PM}</td><td>${r.HC_T}</td><td>${r.HC_2}</td><td>${r.DR.toFixed(1)}</td><td>${r.TB}</td><td data-sort="${r.BR}">${brStr}</td><td>${r.NS}</td><td data-sort="${r.ST_str}"><span style="color:${stColor};font-weight:bold;">${r.ST_str}</span></td><td style="background:#fafafa; font-weight:bold; color:#4a148c; font-size:1.1em;" data-sort="${r.cssFinal}">${r.cssFinal.toFixed(2)}</td></tr>`; }); tableHTML += `</tbody></table>`;
    return `<div class="academic-brief" style="max-width: 1100px; padding: 60px 80px; page-break-before: always;"><div class="brief-section-title" style="margin-top:20px;">Master CSS v2a Evaluation Matrix</div>${tableHTML}</div>`;
}
window.runConsolidator = function(mode) { if(mode === 'matrix') { document.title = "Master_CSS_Matrix"; document.getElementById('report-container').innerHTML = getMatrixHTML(null) + `__APPENDIX_A_HTML__`; setTimeout(() => { if(window.init) window.init(); }, 100); return; } }
</script>""".replace('__APPENDIX_A_HTML__', APPENDIX_A_HTML)
CONSOLIDATOR_HTML = f"""<div class="no-print consol-panel"><h2 style="color:#4a148c; margin-top:0;">The Omni-Proof Consolidator</h2><div style="display:flex; justify-content:center; gap:15px; flex-wrap:wrap; margin-bottom:15px;"><div style="flex:1; min-width:250px; text-align:left;"><label style="font-size:12px; font-weight:bold; color:#4a148c;">Select Tester</label><select id="testerSelect" style="width:100%; padding:8px; border:1px solid #7b1fa2; border-radius:4px;"><option value="">-- Choose One --</option></select></div><div style="flex:1; min-width:250px; text-align:left;"><label style="font-size:12px; font-weight:bold; color:#4a148c;">Select Target Ancestor</label><select id="ancestorSelect" style="width:100%; padding:8px; border:1px solid #7b1fa2; border-radius:4px;"><option value="">-- Choose Target --</option></select></div></div><button class="consol-btn" style="background:#2e7d32; margin-left:10px;" onclick="runConsolidator('matrix')">📊 Generate CSS Matrix</button></div><div id="report-container"></div>"""

# 🌟 4. THE 4 CORE TOOLS
BIO_TMPL = r"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Biological Proof Register</title><style>__CSS_BASE__</style></head><body><div class="wrap"><h1 class="centerline no-print" style="margin-top:30px; color:#006064;">📜 Biological Proof Register</h1><div id="nav-slot" class="no-print">__STATS_BAR____NAV_HTML__</div><div class="proof-card"><div id="proof-ui"></div><div id="proof-result"></div></div>__LEGAL_FOOTER__</div>
<script>__JS_GLOBALS__;
document.addEventListener('DOMContentLoaded', function() {
    const ancKeys = Object.keys(DATA.ancestors).sort((a,b) => DATA.ancestors[a].name.localeCompare(DATA.ancestors[b].name));
    let sel = '<select id="ancSelect" onchange="renderBio()" style="padding:12px; width:100%; border:2px solid #006064; border-radius:4px; font-size:16px; margin-bottom:20px;"><option value="">-- Select an Ancestral Line to Verify --</option>';
    ancKeys.forEach(k => { sel += `<option value="${k}">${DATA.ancestors[k].name} (${DATA.ancestors[k].matches} proper matches)</option>`; });
    sel += '</select>'; document.getElementById('proof-ui').innerHTML = sel;
    window.renderBio = function() {
        let k = document.getElementById('ancSelect').value; if(!k) { document.getElementById('proof-result').innerHTML = ''; return; } let a = DATA.ancestors[k];
        let html = `<div style="background:#e0f7fa; padding:20px; border-radius:8px; border-left:5px solid #00acc1; margin-bottom:20px;"><h2 style="margin-top:0; color:#006064;">Biological Proof: ${a.name}</h2><p style="font-size:16px;"><strong>Forensic Validation:</strong> <span class="badge badge-${a.badge.toLowerCase()}">${a.badge} Standard</span></p><p style="font-size:16px;"><strong>Integrity Score:</strong> ${a.integrity}% (Verified by ${a.testers} independent tester kits)</p><p style="font-size:16px;"><strong>Total Evidence:</strong> ${a.cm} cM of shared autosomal DNA across ${a.matches} matching paths.</p></div><table class="brief-table"><thead><tr><th>Matching Kit</th><th>Shared cM</th><th>Documented Lineage Path</th></tr></thead><tbody>`;
        let matches = DB.filter(m => m.ancestor === a.name).sort((x,y) => parseInt(y.cm||0) - parseInt(x.cm||0));
        matches.forEach(m => { html += `<tr><td style="width:25%;"><strong>${m.participant}</strong></td><td style="width:15%; color:#006064; font-weight:bold;">${m.cm} cM</td><td style="font-size:0.9em;color:#555;">${m.lineage.replace(/->/g, '&rarr;')}</td></tr>`; });
        html += '</tbody></table>'; document.getElementById('proof-result').innerHTML = html;
    }
});
</script></body></html>"""

PROOF_TMPL = r"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Lineage Proof Engine</title><style>__CSS_BASE__</style></head><body><div class="wrap"><h1 class="centerline no-print" style="margin-top:30px; color:#4a148c;">🧬 Lineage Proof Engine</h1><div id="nav-slot" class="no-print">__STATS_BAR____NAV_HTML__</div><div class="proof-card"><div id="proof-ui"></div><div id="proof-result"></div></div>__LEGAL_FOOTER__</div>
<script>__JS_GLOBALS__;
document.addEventListener('DOMContentLoaded', function() {
    const partKeys = Object.keys(DATA.participants).sort((a,b) => DATA.participants[a].sort_key.localeCompare(DATA.participants[b].sort_key));
    let sel = '<select id="partSelect" onchange="renderProof()" style="padding:12px; width:100%; border:2px solid #4a148c; border-radius:4px; font-size:16px; margin-bottom:20px;"><option value="">-- Select a Study Participant --</option>';
    partKeys.forEach(k => { sel += `<option value="${k}">${DATA.participants[k].name}</option>`; });
    sel += '</select>'; document.getElementById('proof-ui').innerHTML = sel;
    window.renderProof = function() {
        let k = document.getElementById('partSelect').value; if(!k) { document.getElementById('proof-result').innerHTML = ''; return; } let p = DATA.participants[k];
        let html = `<div style="background:#f3e5f5; padding:20px; border-radius:8px; border-left:5px solid #ab47bc; margin-bottom:20px;"><h2 style="margin-top:0; color:#4a148c;">Tester Profile: ${p.name}</h2><p style="font-size:16px;"><strong>Status:</strong> <span class="badge" style="background:#4a148c; color:white;">${p.badge}</span></p><p style="font-size:16px;"><strong>Total Evidence Mass:</strong> ${p.cm} cM corroborating ${p.matches} node connections.</p></div><h3 style="color:#4a148c; border-bottom:2px solid #ccc; padding-bottom:5px;">Confirmed Ancestral Intersections</h3><table class="brief-table"><thead><tr><th>Target Node</th><th>Shared cM</th><th>Participant's Triangulation Path</th></tr></thead><tbody>`;
        let matches = DB.filter(m => m.participant === p.name).sort((x,y) => parseInt(y.cm||0) - parseInt(x.cm||0));
        matches.forEach(m => { html += `<tr><td style="width:30%;"><strong>${m.ancestor}</strong></td><td style="width:10%; color:#4a148c; font-weight:bold;">${m.cm} cM</td><td style="font-size:0.9em;color:#555;">${m.lineage.replace(/->/g, '&rarr;')}</td></tr>`; });
        html += '</tbody></table>'; document.getElementById('proof-result').innerHTML = html;
    }
});
</script></body></html>"""

DOSS_TMPL = r"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Forensic Dossier</title><style>__CSS_BASE__</style></head><body><div class="wrap"><h1 class="centerline no-print" style="margin-top:30px; color:#004d40;">📁 Forensic Dossier</h1><div id="nav-slot" class="no-print">__STATS_BAR____NAV_HTML__</div><div class="proof-card"><div id="dos-ui" class="no-print"></div><div id="report-stack"></div></div>__LEGAL_FOOTER__</div>
<script>__JS_GLOBALS__;
document.addEventListener('DOMContentLoaded', function() {
    const dosKeys = Object.keys(DATA.participants).sort((a,b) => DATA.participants[a].sort_key.localeCompare(DATA.participants[b].sort_key));
    let sel = '<select id="dosSelect" onchange="renderDossier()" style="padding:12px; width:100%; border:2px solid #004d40; border-radius:4px; font-size:16px; margin-bottom:20px;"><option value="">-- Select Kit to Generate Formal Dossier --</option>';
    dosKeys.forEach(k => { sel += `<option value="${k}">${DATA.participants[k].name}</option>`; });
    sel += '</select>'; document.getElementById('dos-ui').innerHTML = sel;
    window.renderDossier = function() {
        let k = document.getElementById('dosSelect').value; if(!k) { document.getElementById('report-stack').innerHTML = ''; return; } let p = DATA.participants[k];
        let matches = DB.filter(m => m.participant === p.name).sort((x,y) => parseInt(y.cm||0) - parseInt(x.cm||0)); let topAnc = matches.length > 0 ? matches[0].ancestor : "None Found";
        let html = `<div style="border: 3px double #004d40; padding: 40px; background: white; margin-top:20px;"><div style="text-align:center; border-bottom:2px solid #004d40; padding-bottom:20px; margin-bottom:30px;"><h1 style="color:#004d40; text-transform:uppercase; margin:0; font-size:28px;">Forensic Evidence Dossier</h1><p style="margin:5px 0 0 0; color:#555; font-style:italic;">Yates DNA Study Lineage Reconstruction</p></div><div style="font-size:16px; line-height:1.8; background:#f4f4f4; padding:20px; border:1px solid #ddd; margin-bottom:30px;"><p style="margin:0;"><strong>SUBJECT IDENTIFIER:</strong> ${p.name}</p><p style="margin:0;"><strong>EVIDENCE INTEGRITY SCORE:</strong> ${p.integrity}%</p><p style="margin:0;"><strong>PRIMARY CORROBORATED NODE:</strong> ${topAnc}</p><hr style="border-top:1px solid #ccc; margin:15px 0;"><p style="margin:0;"><strong>EXECUTIVE SUMMARY:</strong> This subject shares ${p.cm} cM of autosomal DNA across ${p.matches} independently verified ancestral nodes within the Yates study. The empirical data confirms the biological validity of the subject's descent pathway.</p></div><h3 style="color:#004d40; text-transform:uppercase; font-size:18px;">Cross-Referenced Match Index</h3><table class="brief-table"><thead><tr><th style="background:#e0f2f1;">Intersected Study Node</th><th style="background:#e0f2f1;">cM</th><th style="background:#e0f2f1;">Documented Route to Node</th></tr></thead><tbody>`;
        matches.forEach(m => { html += `<tr><td style="width:30%;"><strong>${m.ancestor}</strong></td><td style="width:10%; color:#004d40; font-weight:bold;">${m.cm}</td><td style="font-size:0.85em;color:#333; line-height:1.4;">${m.lineage.replace(/->/g, '<br>&uarr;')}</td></tr>`; });
        html += '</tbody></table></div>'; document.getElementById('report-stack').innerHTML = html;
    }
});
</script></body></html>"""

BUST_TMPL = r"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Brick Wall Buster</title><style>__CSS_BASE__</style></head><body><div class="wrap"><h1 class="centerline no-print" style="margin-top:30px; color:#e65100;">🧱 Brick Wall Buster</h1><div id="nav-slot" class="no-print">__STATS_BAR____NAV_HTML__</div><div class="proof-card"><div id="cluster-table-div"></div></div>__LEGAL_FOOTER__</div>
<script>__JS_GLOBALS__;
document.addEventListener('DOMContentLoaded', function() {
    let html = `<div style="background:#fff8e1; padding:20px; border-radius:8px; border-left:5px solid #f57f17; margin-bottom:30px;"><h2 style="margin-top:0; color:#e65100;">Cluster Analysis Engine</h2><p style="font-size:16px;">This algorithm maps independent DNA kits that converge on the same ancestral node. Nodes with multiple distinct testers and high genetic integrity are flagged as "Hot Paths" to break genealogical brick walls.</p></div><table class="brief-table sortable"><thead><tr><th style="background:#fff8e1;">Emergent Ancestral Node</th><th style="background:#fff8e1; text-align:center;">Independent Testers</th><th style="background:#fff8e1; text-align:center;">Total cM Mass</th><th style="background:#fff8e1; text-align:center;">Cluster Integrity</th></tr></thead><tbody>`;
    let clusters = Object.values(DATA.ancestors).filter(a => a.matches >= 2).sort((x,y) => y.integrity - x.integrity);
    clusters.forEach(c => {
        let badge = `<span class="badge badge-${c.badge.toLowerCase()}">${c.badge}</span>`;
        html += `<tr><td><strong style="font-size:1.1em; color:#e65100;">${c.name}</strong><br><div style="margin-top:5px;">${badge}</div></td><td style="text-align:center; font-size:1.2em; font-weight:bold;">${c.testers}</td><td style="text-align:center;">${c.cm} cM</td><td style="text-align:center; color:green; font-weight:bold;">${c.integrity}%</td></tr>`;
    });
    html += '</tbody></table>'; document.getElementById('cluster-table-div').innerHTML = html;
});
</script></body></html>"""

print("✅ Cell 4 (Pristine Master Merge + ALL CSS Vars Patched) Loaded Successfully.")

      [CELL 4] TEMPLATE LIBRARY LOADING (Pristine UI Restored)...
✅ Cell 4 (Pristine Master Merge + ALL CSS Vars Patched) Loaded Successfully.


In [42]:
# @title [CELL 5] Core Publisher & FTP Uploader (Patch 6: Admin Views & GEDmatch)
def run_publisher():
    print("="*60)
    print("      [CELL 5] PUBLISHER STARTING (Admin Views & GEDmatch)...")
    print("="*60)

    import os, re, pytz, json, csv
    import pandas as pd
    from datetime import datetime
    from google.colab import userdata
    from ftplib import FTP_TLS

    try:
        HOST = os.environ.get("FTP_HOST") or userdata.get("FTP_HOST")
        USER = os.environ.get("FTP_USER") or userdata.get("FTP_USER")
        PASS = os.environ.get("FTP_PASS") or userdata.get("FTP_PASS")
    except Exception as e:
        return print(f"❌ Credential Error: {e}")

    CSV_DB = "engine_database.csv"
    KEY_FILE = "match_to_unmasked.csv"

    if not os.path.exists(CSV_DB): return print("❌ ERROR: engine_database.csv not found.")

    df = pd.read_csv(CSV_DB, encoding="iso-8859-15")
    df.fillna('', inplace=True)
    df.replace('nan', '', inplace=True)

    # Privacy Truncation
    def shorten_name(full_name):
        if pd.isna(full_name) or str(full_name).lower() == 'nan' or not str(full_name).strip(): return "Unknown"
        s = re.sub(r'\[.*?\]', '', str(full_name)).strip()
        parts = s.split()
        if len(parts) <= 1: return s
        suffix = ""
        if parts[-1].lower() in ['jr', 'jr.', 'sr', 'sr.', 'iii', 'iv', 'v', 'md', 'm.d.', 'esq', 'esq.']:
            suffix = " " + parts.pop()
        if len(parts) == 1: return parts[0] + suffix
        initials = "".join([p[0].upper() + "." for p in parts[:-1]])
        return f"{initials} {parts[-1]}{suffix}"

    db_ids = {}
    for _, r in df.iterrows():
        c = str(r.get('Tester_Code', '')).strip().lower()
        i = str(r.get('Tester_ID', '')).replace('I', '').strip()
        if c and i and i != 'nan': db_ids[c] = i

    tester_auth = []
    if os.path.exists(KEY_FILE):
        with open(KEY_FILE, 'r', errors='replace') as f:
            reader = csv.reader(f)
            for i, row in enumerate(reader):
                if len(row) >= 2:
                    if i == 0 and ("tester" in row[0].lower() or "masked" in row[0].lower() or "code" in row[0].lower()): continue
                    code = row[0].strip().lower()
                    name = row[1].strip()
                    tid = re.sub(r'[^0-9]', '', row[2].strip() if len(row) > 2 else "")
                    if code in db_ids: tid = db_ids[code]
                    tester_auth.append({'Kit_Code': code, 'Tester_Name': shorten_name(name), 'Tester_ID': tid})

    df_testers = pd.DataFrame(tester_auth)
    if df_testers.empty:
        fallback = []
        for kcode, grp in df.groupby('Tester_Code'):
            tid = str(grp.iloc[0]['Tester_ID']).replace('I','').strip()
            fallback.append({'Kit_Code': kcode, 'Tester_Name': shorten_name(grp.iloc[0]['Tester_Name']), 'Tester_ID': tid})
        df_testers = pd.DataFrame(fallback)

    df['Kit_Name'] = df.apply(
        lambda r: f"{shorten_name(r['Tester_Name'])} [I{re.sub(r'[^0-9]','',str(r['Tester_ID']))}]"
        if pd.notna(r['Tester_ID']) and re.sub(r'[^0-9]','',str(r['Tester_ID']))
        else f"{shorten_name(r['Tester_Name'])} [{r['Tester_Code']}]", axis=1)

    df.rename(columns={"Authority_Directory_Label": "Dir_Label", "Authority_FirstAncestor_alpha": "Alpha_Key", "Tester_Code": "Kit_Code", "Match_Lineage": "Lineage", "Match_Path_IDs": "s_ids"}, inplace=True)

    def normalize_id(val): return f"I{str(val).replace('@', '').strip()}" if str(val).replace('@', '').strip().isdigit() else str(val).replace('@', '').strip()

    df['search_ids'] = df['s_ids']
    df['search_names'] = df['Lineage'].astype(str).str.replace(' -> ', '|')
    df['t_names'] = df['Tester_Lineage'].astype(str).str.replace(' -> ', '|')
    df['t_ids'] = df['Tester_Path_IDs'].astype(str).str.replace(',', '|')

    est = pytz.timezone('US/Eastern')
    timestamp = datetime.now(est).strftime("%B %d, %Y %-I:%M %p EST")
    stats_bar_full = f'<div style="background:#f4f4f4;border-top:1px solid #ddd;border-bottom:1px solid #ddd;font-family:sans-serif;font-size:12px;color:#555;padding:8px 15px;text-align:center;margin-bottom:0;"><strong>Study Data Current As Of:</strong> {timestamp} | <strong>Total Autosomal matches:</strong> {len(df):,}</div>'

    current_year = str(datetime.now(est).year)
    LEGAL_FOOTER_TMPL = r"""<div class="legal-footer no-print" style="margin-top:50px;padding:20px;background:#f4f4f4;border-top:1px solid #ddd;text-align:center;color:#666;font-family:sans-serif;font-size:0.85em;clear:both;"><p style="margin-bottom:5px;font-size:1.1em;color:#333;"><strong>&copy; __YEAR__ Ronald Eugene Yates. All Rights Reserved.</strong></p><p style="margin-bottom:5px;">Generated by <em>The Forensic Genealogy Publisher&trade;</em></p><p style="font-style:italic;color:#888;margin-bottom:0;max-width:800px;margin-left:auto;margin-right:auto;">The terms "Forensic Handshake", "Brick Wall Buster", and "Collateral Saturation" are trademarks of Ronald Eugene Yates.</p></div>"""
    LEGAL_FOOTER = LEGAL_FOOTER_TMPL.replace('__YEAR__', current_year)

    REGISTER_CSS_CENTERED = "<style>.table-scroll-wrapper{overflow-x:auto; background:white; padding:20px; border-radius:8px; box-shadow:0 4px 6px rgba(0,0,0,0.1); max-width:1400px; margin:20px auto;}</style>"

    # Inject GEDmatch into the Global Navigation
    global NAV_HTML
    if "gedmatch_integration.shtml" not in NAV_HTML:
        NAV_HTML = NAV_HTML.replace('<li><a href="/ons-study/share_dna.shtml"', '<li><a href="/ons-study/gedmatch_integration.shtml" style="color:#81d4fa;">GEDmatch</a></li><li><a href="/ons-study/share_dna.shtml"')

    def make_page(title, content, nav_b, bar, extra_css=""):
        s_info = SITE_INFO if nav_b else ""
        return f"<!DOCTYPE html>\n<html lang=\"en\">\n<head>\n<meta charset=\"UTF-8\">\n<title>{title}</title>\n<link rel=\"stylesheet\" href=\"partials_unified.css\">\n<link rel=\"stylesheet\" href=\"dna_tree_styles.css\">\n{extra_css}\n</head>\n<body id=\"top\">\n<div class=\"wrap\">\n<h1 class=\"centerline no-print\">{title}</h1>\n<div id=\"nav-slot\">{bar}{NAV_HTML}</div>\n{s_info}{content}\n</div>\n{LEGAL_FOOTER}{JS_CORE}{BTT_BTN}\n</body>\n</html>"

    def get_sort_key(name):
        if pd.isna(name) or not name: return "zzz"
        s = re.sub(r'\[.*?\]', '', str(name))
        cleaned = re.sub(r'\b(jr\.?|sr\.?|iii|iv|v|md|m\.d\.|esq\.?)\b', '', s, flags=re.IGNORECASE)
        parts = re.split(r'\bnee\b|\bnée\b', cleaned.lower())[0].replace(',', '').replace('.', '').strip().split()
        return parts[-1] if parts else "zzz"

    match_counts = df.groupby('Kit_Code').size().reset_index(name='Match_Count')
    part_stats = pd.merge(df_testers, match_counts, on='Kit_Code', how='left')
    part_stats['Match_Count'] = part_stats['Match_Count'].fillna(0).astype(int)
    part_stats['Sort_Key'] = part_stats['Tester_Name'].apply(get_sort_key)

    def make_admin_row(r):
        tid = str(r['Tester_ID']).strip()
        tname, kcode, mc_val = str(r["Tester_Name"]), str(r["Kit_Code"]), r['Match_Count']
        t_link = f'<a href="https://yates.one-name.net/tng/getperson.php?personID=I{tid}&tree=tree1" target="_blank" style="color:#00838f;text-decoration:underline;font-weight:bold;">{tname}</a>' if tid and tid != 'nan' else f'<b style="color:#333;">{tname}</b>'
        tid_display = f" <span style='color:#777;font-size:0.85em;'>[I{tid}]</span>" if tid and tid != 'nan' else ""
        mc_str = f"<span style='color:#d32f2f;font-weight:bold;'>0</span>" if mc_val == 0 else str(mc_val)
        return f"<tr><td data-sort='{r['Sort_Key']}'>{t_link}{tid_display}<br><span style='color:#666;font-size:0.85em;'>Kit: {kcode}</span></td><td style='text-align:center;font-size:1.1em;vertical-align:middle;'>{mc_str}</td></tr>"

    # 🌟 NEW DUAL SORTING FOR ADMIN HUB 🌟
    admin_rows_az = [make_admin_row(r) for _, r in part_stats.sort_values(['Sort_Key', 'Tester_Name'], ascending=[True, True]).iterrows()]
    admin_rows_count = [make_admin_row(r) for _, r in part_stats.sort_values(['Match_Count', 'Sort_Key'], ascending=[True, True]).iterrows()]

    anc_data = {}; part_data = {}
    for lbl, grp in df.groupby('Dir_Label'):
        if len(grp)<2: continue
        unique_t = len(grp['Kit_Name'].unique())
        integ = min(100, (len(grp)*2) + (unique_t*10))
        anc_data[grp.iloc[0]['Alpha_Key']] = {"name": lbl, "matches": len(grp), "cm": int(grp['cM'].sum()), "badge": "Platinum" if len(grp)>=30 else "Gold" if len(grp)>=15 else "Silver" if len(grp)>=5 else "Bronze", "list_data": grp['Kit_Name'].value_counts().head(3).to_dict(), "verdict": "Verified.", "integrity": integ, "testers": unique_t}

    for kname, grp in df.groupby('Kit_Name'):
        dir_lbl = grp.iloc[0]['Dir_Label']
        same_dir = df[df['Dir_Label'] == dir_lbl] if pd.notna(dir_lbl) else pd.DataFrame()
        integ = min(100, len(same_dir) * 5)
        part_data[kname] = {"name": kname, "sort_key": get_sort_key(kname), "matches": len(grp), "cm": int(grp['cM'].sum()), "badge": "Keystone Tester" if len(grp)>=15 else "Study Participant", "list_data": grp['Dir_Label'].value_counts().head(3).to_dict(), "verdict": f"Verified matches across {len(grp['Dir_Label'].unique())} ancestral lines.", "integrity": integ, "testers": 1}

    smart_json = json.dumps({"ancestors": anc_data, "participants": part_data})
    db_json = df[['Dir_Label', 'Kit_Name', 'cM', 'Match_ID', 'Lineage', 'search_ids', 'search_names', 't_names', 't_ids', 'Tester_ID']].rename(columns={'Dir_Label':'ancestor', 'Kit_Name':'participant', 'cM':'cm', 'Match_ID':'id', 'Lineage':'lineage', 'Tester_ID':'tester_id'}).to_json(orient='records')
    JS_GLOBALS = f"const DATA={smart_json}; const DB={db_json};"

    pages_to_upload = {}

    pages_to_upload["proof_consolidator.html"] = make_page("Proof Consolidator", CONSOLIDATOR_HTML, False, stats_bar_full, extra_css=CONSOLIDATOR_CSS).replace('</body>', CONSOLIDATOR_JS.replace('__JS_GLOBALS__', JS_GLOBALS) + '</body>')
    pages_to_upload["biological_proof.html"] = BIO_TMPL.replace('__CSS_BASE__', CSS_BASE).replace('__STATS_BAR__', stats_bar_full).replace('__NAV_HTML__', NAV_HTML).replace('__JS_GLOBALS__', JS_GLOBALS).replace('__LEGAL_FOOTER__', LEGAL_FOOTER)
    pages_to_upload["lineage_proof.html"] = PROOF_TMPL.replace('__CSS_BASE__', CSS_BASE).replace('__STATS_BAR__', stats_bar_full).replace('__NAV_HTML__', NAV_HTML).replace('__JS_GLOBALS__', JS_GLOBALS).replace('__LEGAL_FOOTER__', LEGAL_FOOTER)
    pages_to_upload["dna_dossier.html"] = DOSS_TMPL.replace('__CSS_BASE__', CSS_BASE).replace('__STATS_BAR__', stats_bar_full).replace('__NAV_HTML__', NAV_HTML).replace('__JS_GLOBALS__', JS_GLOBALS).replace('__LEGAL_FOOTER__', LEGAL_FOOTER)
    pages_to_upload["brick_wall_buster.shtml"] = BUST_TMPL.replace('__CSS_BASE__', CSS_BASE).replace('__STATS_BAR__', stats_bar_full).replace('__NAV_HTML__', NAV_HTML).replace('__JS_GLOBALS__', JS_GLOBALS).replace('__LEGAL_FOOTER__', LEGAL_FOOTER)
    pages_to_upload["contents.shtml"] = make_page("Yates Study User Guide", CONTENTS_CONTENT, False, stats_bar_full, extra_css=CONTENTS_CSS)
    pages_to_upload["share_dna.shtml"] = make_page("Share Your Ancestry DNA Matches", SHARE_CONTENT, False, stats_bar_full, extra_css=SHARE_CSS)
    pages_to_upload["subscribe.shtml"] = make_page("Join the Yates Research Community", SUBSCRIBE_CONTENT, False, stats_bar_full, extra_css=SUBSCRIBE_CSS)
    pages_to_upload["dna_theory_of_the_case.htm"] = make_page("The Yates DNA Strategy", THEORY_CONTENT, False, stats_bar_full, extra_css=THEORY_CSS)
    pages_to_upload["data_glossary.shtml"] = make_page("Data Glossary", GLOSSARY_CONTENT, False, stats_bar_full, extra_css=GLOSS_CSS)

    # 🌟 NEW GEDMATCH HUB CREATION 🌟
    GEDMATCH_CONTENT = r"""<div class="proof-card"><h2 style="color:#01579b; border-bottom:2px solid #03a9f4; padding-bottom:10px;">GEDmatch Integration & Segment Mapping</h2><p style="font-size:1.1em; line-height:1.6; color:#333;">This module handles the advanced import and visualization of autosomal segment data exported directly from GEDmatch. By integrating segment-level data, we mathematically cross-reference precise overlapping chromosome coordinates to validate our node-based connections.</p><div style="text-align:center; margin-top:40px; padding:20px; background:#e1f5fe; border:1px dashed #b3e5fc; border-radius:8px;"><p style="color:#0277bd; font-weight:bold; font-size:1.2em;">Segment Parser Offline</p><p style="color:#555;">The automated chromosome mapping utility is currently undergoing maintenance. Please return later.</p></div></div>"""
    pages_to_upload["gedmatch_integration.shtml"] = make_page("GEDmatch Hub", GEDMATCH_CONTENT, False, stats_bar_full)

    # 🌟 ADMIN HUB UPGRADE (Dual Views + GEDmatch Card) 🌟
    admin_content = f"""
    <div class="dashboard-grid">
        <a href="ons_yates_dna_register.shtml" class="dash-card"><span class="dash-icon">📋</span><span class="dash-title">DNA Register</span></a>
        <a href="dna_network.shtml" class="dash-card"><span class="dash-icon">🕸️</span><span class="dash-title">DNA Network</span></a>
        <a href="proof_consolidator.html" class="dash-card" style="border-color:#4a148c; background:#f3e5f5;"><span class="dash-icon">🎓</span><span class="dash-title" style="color:#4a148c;">Proof Consolidator</span></a>
        <a href="biological_proof.html" class="dash-card"><span class="dash-icon">📜</span><span class="dash-title">Bio Proof</span></a>
        <a href="lineage_proof.html" class="dash-card"><span class="dash-icon">🧬</span><span class="dash-title">Proof Engine</span></a>
        <a href="dna_dossier.html" class="dash-card"><span class="dash-icon">📁</span><span class="dash-title">Forensic Dossier</span></a>
        <a href="gedmatch_integration.shtml" class="dash-card" style="border-color:#0277bd; background:#e1f5fe;"><span class="dash-icon">🔬</span><span class="dash-title" style="color:#01579b;">GEDmatch Hub</span></a>
        <a href="engine_database.csv" download class="dash-card" style="border-color:#b71c1c; background:#ffebee;"><span class="dash-icon">💾</span><span class="dash-title" style="color:#c62828;">Download CSV Database</span></a>
    </div>

    <div class="audit-table-wrapper">
        <h2 style="color:#004d40;border-bottom:2px solid #004d40;padding-bottom:10px;margin-top:0;">Participant Activity Report - {len(part_stats)} Testers</h2>

        <div style="text-align:center; margin-bottom:20px; font-family:sans-serif;">
            <button onclick="document.getElementById('table-az').style.display='block'; document.getElementById('table-count').style.display='none'; this.style.background='#006064'; this.style.color='white'; document.getElementById('btn-count').style.background='#e0f7fa'; document.getElementById('btn-count').style.color='#006064';" id="btn-az" style="padding:10px 20px; cursor:pointer; font-weight:bold; background:#006064; color:white; border:1px solid #00acc1; border-radius:4px;">Sort A-Z (Default)</button>
            <button onclick="document.getElementById('table-count').style.display='block'; document.getElementById('table-az').style.display='none'; this.style.background='#006064'; this.style.color='white'; document.getElementById('btn-az').style.background='#e0f7fa'; document.getElementById('btn-az').style.color='#006064';" id="btn-count" style="padding:10px 20px; cursor:pointer; font-weight:bold; background:#e0f7fa; color:#006064; border:1px solid #00acc1; border-radius:4px; margin-left:10px;">Sort by Match Count (Low to High)</button>
        </div>

        <div id="table-az" style="max-height:600px;overflow-y:auto;border:1px solid #ddd;background:#fafafa; display:block;">
            <table style="width:100%;border-collapse:collapse;">
                <thead><tr><th style="background:#004d40;color:white;padding:12px;text-align:left;position:sticky;top:0;">Participant Kit</th><th style="background:#004d40;color:white;padding:12px;text-align:center;position:sticky;top:0;">Matches</th></tr></thead>
                <tbody>{''.join(admin_rows_az)}</tbody>
            </table>
        </div>

        <div id="table-count" style="max-height:600px;overflow-y:auto;border:1px solid #ddd;background:#fafafa; display:none;">
            <table style="width:100%;border-collapse:collapse;">
                <thead><tr><th style="background:#004d40;color:white;padding:12px;text-align:left;position:sticky;top:0;">Participant Kit</th><th style="background:#004d40;color:white;padding:12px;text-align:center;position:sticky;top:0;">Matches</th></tr></thead>
                <tbody>{''.join(admin_rows_count)}</tbody>
            </table>
        </div>
    </div>
    """

    pages_to_upload["research_admin.html"] = make_page("Yates Research Admin Hub", admin_content, False, stats_bar_full, extra_css=ADMIN_CSS + "<style>.dashboard-grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(200px,1fr));gap:20px;margin:30px auto;max-width:1200px}.dash-card{background:white;padding:20px;border-radius:8px;text-align:center;box-shadow:0 4px 6px rgba(0,0,0,0.1);text-decoration:none;color:#333;border:1px solid #ddd; transition:transform 0.2s;}.dash-card:hover{transform:translateY(-5px);border-color:#006064;background:#e0f7fa}.dash-icon{font-size:40px;margin-bottom:10px;display:block}.dash-title{font-weight:bold;font-size:1.1em;color:#006064}.audit-table-wrapper{background:white;padding:25px;border-radius:8px;box-shadow:0 4px 6px rgba(0,0,0,0.1);max-width:1400px;margin:0 auto} td{padding:10px;border-bottom:1px solid #eee}</style>")

    df_valid = df[df['Dir_Label'] != 'No Matches'].copy()
    df_valid['sort_key'] = df_valid['Kit_Name'].apply(get_sort_key)
    mc = df_valid['Dir_Label'].value_counts()

    def format_reg(r):
        m_id = str(r.get("Match_ID", ""))
        m_name = str(r.get("Match_Name", ""))
        d_label = str(r.get("Dir_Label", "")).split('(')[0].strip()
        lin_len = len(str(r.get("Lineage", "")).split("->"))
        kit_name = str(r["Kit_Name"])
        cm_val = str(r["cM"])
        text = f"<b>{kit_name}</b> is a {cm_val} cM match to <a href='https://yates.one-name.net/tng/verticalchart.php?personID={normalize_id(m_id)}&tree=tree1&parentset=0&display=vertical&generations=15' target='_blank'><b>{m_name}</b></a> via {d_label} back {lin_len} generations."
        if mc.get(r['Dir_Label'], 0) == 1:
            return f"<div style='background-color: #fffde7; padding: 12px; margin: -12px; border-left: 5px solid #fbc02d;'>{text} <span style='float:right; font-size:0.85em; color:#e65100; font-weight:bold; background:#fff8e1; padding:3px 8px; border-radius:4px; border:1px solid #ffe082;'>🌟 Singleton Line</span></div>"
        return f"<div style='padding: 12px; margin: -12px;'>{text}</div>"

    def format_tree(r):
        m_id = str(r.get("Match_ID", ""))
        m_name = str(r.get("Match_Name", ""))
        lin_str = str(r.get("Lineage", ""))
        kit_name = str(r["Kit_Name"])
        if m_name in lin_str:
            linked_lin = lin_str.replace(m_name, f'<a href="https://yates.one-name.net/tng/verticalchart.php?personID={normalize_id(m_id)}&tree=tree1&parentset=0&display=vertical&generations=15" target="_blank" style="color:#006064;text-decoration:none;font-weight:bold;">{m_name}</a>')
        else:
            linked_lin = lin_str
        text = f"<b style='color:#4a148c;'>{kit_name}</b>: {linked_lin}"
        if mc.get(r['Dir_Label'], 0) == 1:
            return f"<div style='background-color: #fffde7; padding: 12px; margin: -12px; border-left: 5px solid #fbc02d;'>{text} <span style='float:right; font-size:0.85em; color:#e65100; font-weight:bold; background:#fff8e1; padding:3px 8px; border-radius:4px; border:1px solid #ffe082;'>🌟 Singleton Line</span></div>"
        return f"<div style='padding: 12px; margin: -12px;'>{text}</div>"

    df_valid['Reg_Narrative'] = df_valid.apply(format_reg, axis=1)
    df_valid['Tree_Narrative'] = df_valid.apply(format_tree, axis=1)

    df_reg_za = df_valid.sort_values(by=['Dir_Label', 'sort_key'], ascending=[False, True]).copy()
    df_reg_za.rename(columns={'Reg_Narrative': 'Participants who tested-Who they matched-Oldest known Yates ancestor'}, inplace=True)
    df_reg_az = df_valid.sort_values(by=['sort_key', 'Dir_Label'], ascending=[True, False]).copy()
    df_reg_az.rename(columns={'Reg_Narrative': 'Participants who tested-Who they matched-Oldest known Yates ancestor'}, inplace=True)

    df_tree_za = df_valid.sort_values(by=['Dir_Label', 'sort_key'], ascending=[False, True]).copy()
    df_tree_za.rename(columns={'Tree_Narrative': 'Visual Lineage Path'}, inplace=True)
    df_tree_az = df_valid.sort_values(by=['sort_key', 'Dir_Label'], ascending=[True, False]).copy()
    df_tree_az.rename(columns={'Tree_Narrative': 'Visual Lineage Path'}, inplace=True)

    toggle_reg_za = f'<div class="no-print" style="text-align:center; margin:15px auto; max-width:1400px; padding:10px; background:#e0f7fa; border:1px solid #b2ebf2; border-radius:4px; font-family:sans-serif; font-size:14px;"><strong>Sort Register:</strong> &nbsp;<span style="color:#006064; font-weight:bold;">By Ancestral Line (Z-A)</span> &nbsp;|&nbsp; <a href="ons_yates_dna_register_participants.shtml" style="color:#00acc1; text-decoration:none;">By Participant (A-Z)</a></div>'
    toggle_reg_az = f'<div class="no-print" style="text-align:center; margin:15px auto; max-width:1400px; padding:10px; background:#e0f7fa; border:1px solid #b2ebf2; border-radius:4px; font-family:sans-serif; font-size:14px;"><strong>Sort Register:</strong> &nbsp;<a href="ons_yates_dna_register.shtml" style="color:#00acc1; text-decoration:none;">By Ancestral Line (Z-A)</a> &nbsp;|&nbsp; <span style="color:#006064; font-weight:bold;">By Participant (A-Z)</span></div>'
    toggle_tree_za = f'<div class="no-print" style="text-align:center; margin:15px auto; max-width:1400px; padding:10px; background:#e0f7fa; border:1px solid #b2ebf2; border-radius:4px; font-family:sans-serif; font-size:14px;"><strong>Sort Trees:</strong> &nbsp;<span style="color:#006064; font-weight:bold;">By Ancestral Line (Z-A)</span> &nbsp;|&nbsp; <a href="just-trees-az.shtml" style="color:#00acc1; text-decoration:none;">By Participant (A-Z)</a></div>'
    toggle_tree_az = f'<div class="no-print" style="text-align:center; margin:15px auto; max-width:1400px; padding:10px; background:#e0f7fa; border:1px solid #b2ebf2; border-radius:4px; font-family:sans-serif; font-size:14px;"><strong>Sort Trees:</strong> &nbsp;<a href="just-trees.shtml" style="color:#00acc1; text-decoration:none;">By Ancestral Line (Z-A)</a> &nbsp;|&nbsp; <span style="color:#006064; font-weight:bold;">By Participant (A-Z)</span></div>'

    pages_to_upload["ons_yates_dna_register.shtml"] = make_page("ONS Yates Study DNA Register", toggle_reg_za + f'<div class="table-scroll-wrapper">{df_reg_za.to_html(columns=["Participants who tested-Who they matched-Oldest known Yates ancestor"], index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, stats_bar_full, extra_css=REGISTER_CSS_CENTERED)
    pages_to_upload["ons_yates_dna_register_participants.shtml"] = make_page("ONS Yates Study DNA Register", toggle_reg_az + f'<div class="table-scroll-wrapper">{df_reg_az.to_html(columns=["Participants who tested-Who they matched-Oldest known Yates ancestor"], index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, stats_bar_full, extra_css=REGISTER_CSS_CENTERED)
    pages_to_upload["just-trees.shtml"] = make_page("Ancestor Register (Trees View)", toggle_tree_za + f'<div class="table-scroll-wrapper">{df_tree_za[["Visual Lineage Path"]].to_html(index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, stats_bar_full, extra_css=REGISTER_CSS_CENTERED)
    pages_to_upload["just-trees-az.shtml"] = make_page("Ancestor Register (Trees View)", toggle_tree_az + f'<div class="table-scroll-wrapper">{df_tree_az[["Visual Lineage Path"]].to_html(index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, stats_bar_full, extra_css=REGISTER_CSS_CENTERED)

    pages_to_upload["admin_singletons.shtml"] = pages_to_upload["ons_yates_dna_register.shtml"]
    pages_to_upload["admin_singletons_participants.shtml"] = pages_to_upload["ons_yates_dna_register_participants.shtml"]
    pages_to_upload["yates_ancestor_register.shtml"] = pages_to_upload["ons_yates_dna_register.shtml"]

    net_buf = []
    for anc, g in sorted(df.groupby('Dir_Label'), key=lambda x: len(x[1]), reverse=True):
        if len(g) < 2 or anc == 'No Matches': continue
        net_buf.append(f"""<details style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;"><span style="font-size:1.1em;">{anc}</span> <span style="float:right;color:#004d40;font-size:0.9em;">Matches: {len(g)} | Total cM: {g['cM'].sum()}</span></summary><div style="padding:15px;"><div style="background:#fffde7;border-left:6px solid #fbc02d;padding:10px;margin-bottom:15px;font-family:sans-serif;color:#333;font-size:0.95em;"><strong>Collateral Saturation Analysis:</strong> Validated by <b>{len(g['Kit_Name'].unique())} independent tester kits</b>.</div><table class="dataframe" border="1" style="width:100%; border-collapse:collapse;"><thead><tr style="text-align:left; background:#eceff1;"><th>Tester Kit</th><th>cM</th><th>Lineage</th></tr></thead><tbody>""")
        for _, r in g.sort_values('cM', ascending=False).iterrows(): net_buf.append(f"<tr><td style='padding:8px; border:1px solid #ddd;'>{r['Kit_Name']}</td><td style='padding:8px; border:1px solid #ddd;'>{r['cM']}</td><td style='padding:8px; border:1px solid #ddd;'>{r['Lineage']}</td></tr>")
        net_buf.append("</tbody></table></div></details>")
    pages_to_upload["dna_network.shtml"] = make_page("Participating DNA Network", f'<div style="margin:20px auto;max-width:1400px;width:95%;">{"".join(net_buf)}</div>', True, stats_bar_full)

    print("\n[LOCAL] Overwriting Database-Driven Pages on disk...")
    for fn, content in pages_to_upload.items():
        if os.path.exists(fn): os.remove(fn)
        with open(fn, "w", encoding="utf-8") as f: f.write(content)

    print("\n[STEP 3] Uploading via FTP to Live Server...")
    try:
        ftps = FTP_TLS()
        ftps.connect(HOST, 21); ftps.auth(); ftps.login(USER, PASS); ftps.prot_p(); ftps.cwd("ons-study")
        upload_count = 0
        for fn in pages_to_upload.keys():
            with open(fn, "rb") as fh: ftps.storbinary(f"STOR {fn}", fh)
            upload_count += 1
            print(f"    [{upload_count}/{len(pages_to_upload)}] 📤 Uploaded HTML: {fn}")

        print(f"\n    [+] Pushing Database File to Server: {CSV_DB}")
        with open(CSV_DB, "rb") as fh: ftps.storbinary(f"STOR {CSV_DB}", fh)
        print(f"    ✅ Uploaded Database: {CSV_DB}")

        ftps.quit()
        print(f"\n🎉 SUCCESS. Uploaded {upload_count} HTML pages and Database directly to the active server.")
    except Exception as e:
        print(f"\n⚠️ FTP SKIP: {e}. ZIP and manual upload required.")

print("✅ Cell 5 (Admin Views & GEDmatch) Loaded.")

✅ Cell 5 (Admin Views & GEDmatch) Loaded.


In [ ]:
# @title [CELL 6] MASTER ORCHESTRATOR (Run This Button)
import os, sys
print("="*60)
print("      MASTER ORCHESTRATOR")
print("      (Running Engine -> Publisher -> Upload)")
print("="*60)

if 'run_engine' not in globals() or 'run_publisher' not in globals():
    print("❌ ERROR: Modules not loaded! Please run the Engine and Publisher setup cells first.")
else:
    print("\n>>> 🚀 PHASE 1: EXECUTING DATA ENGINE...")
    try:
        run_engine()
        print("✅ PHASE 1 COMPLETE.")

        print("\n>>> 🌐 PHASE 2: EXECUTING PUBLISHER & UPLOAD...")
        run_publisher()
        print("✅ PHASE 2 COMPLETE.")

        print("\n" + "="*60)
        print("      🏆 MASTER PIPELINE SUCCESSFUL")
        print("="*60)
    except Exception as e:
        print(f"\n❌ CRITICAL FAILURE: {e}")

      MASTER ORCHESTRATOR
      (Running Engine -> Publisher -> Upload)

>>> 🚀 PHASE 1: EXECUTING DATA ENGINE...
      [CELL 3] ENGINE STARTING (V123 - DEEP RADAR)...

[STEP 1] Resolving Files (Local Priority)...
    ✅ Found match_to_unmasked.csv locally. Skipping FTP download.
    👉 Source GEDCOM: yates_study_2025.ged

[STEP 2] Loading Tester Authority CSV...

[STEP 3] Parsing GEDCOM for Study| Tags & Lineages...

[STEP 4] Constructing Database...

[SUCCESS] Engine V123 Complete. Saved 1713 verified matches to engine_database.csv.
✅ PHASE 1 COMPLETE.

>>> 🌐 PHASE 2: EXECUTING PUBLISHER & UPLOAD...
      [CELL 5] PUBLISHER STARTING (Admin Views & GEDmatch)...

[LOCAL] Overwriting Database-Driven Pages on disk...

[STEP 3] Uploading via FTP to Live Server...
    [1/20] 📤 Uploaded HTML: proof_consolidator.html
    [2/20] 📤 Uploaded HTML: biological_proof.html
    [3/20] 📤 Uploaded HTML: lineage_proof.html
    [4/20] 📤 Uploaded HTML: dna_dossier.html
    [5/20] 📤 Uploaded HTML: brick_wall

In [33]:
# @title [CELL The Time Machine (Archiver + Dropbox Sync)]
import zipfile
import os
import pytz
import time
from datetime import datetime
from google.colab import files
from google.colab import userdata

# --- 1. INSTALL DROPBOX (IF MISSING) ---
try:
    import dropbox
    from dropbox.exceptions import AuthError
except ImportError:
    os.system('pip install dropbox')
    import dropbox
    from dropbox.exceptions import AuthError

def run_archiver():
    print("="*60)
    print("      [CELL 6] MANUAL ARCHIVER + DROPBOX SYNC")
    print("="*60)

    # --- 2. CREATE ZIP (SAFE MODE) ---
    # We explicitly exclude .zip to prevent "Zip Bombs"
    extensions = ('.csv', '.shtml', '.html', '.json', '.js', '.css')
    files_to_pack = [f for f in os.listdir('.') if f.lower().endswith(extensions) and "sample_data" not in f]

    if not files_to_pack:
        print("❌ No generated files found! Run the Publisher (Cell 4) first.")
        return

    est = pytz.timezone('US/Eastern')
    timestamp = datetime.now(est).strftime("%Y-%m-%d_%H%M")
    zip_name = f"Yates_Study_Backup_{timestamp}.zip"

    print(f"📦 Compressing {len(files_to_pack)} files into {zip_name}...")
    try:
        with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zf:
            for file in files_to_pack:
                zf.write(file)
        print(f"    ✅ Archive Created: {zip_name} ({os.path.getsize(zip_name)/1024:.1f} KB)")
    except Exception as e:
        print(f"    ❌ Compression Failed: {e}")
        return

    # --- 3. FTP UPLOAD (BACKUPS FOLDER) ---
    print("\n[STEP 2] Uploading to Web Server (FTP)...")
    try:
        from ftplib import FTP_TLS
        HOST = os.environ.get("FTP_HOST") or userdata.get("FTP_HOST")
        USER = os.environ.get("FTP_USER") or userdata.get("FTP_USER")
        PASS = os.environ.get("FTP_PASS") or userdata.get("FTP_PASS")

        ftps = FTP_TLS()
        ftps.connect(HOST, 21); ftps.auth(); ftps.login(USER, PASS); ftps.prot_p()

        try:
            ftps.cwd("/ons-study/backups")
        except:
            try:
                ftps.mkd("/ons-study/backups")
                ftps.cwd("/ons-study/backups")
            except:
                pass

        with open(zip_name, "rb") as fh:
            ftps.storbinary(f"STOR {zip_name}", fh)
        print(f"    ✅ FTP Success: /ons-study/backups/{zip_name}")
        ftps.quit()
    except Exception as e:
        print(f"    ⚠️ FTP Upload skipped: {e}")

    # --- 4. DROPBOX SYNC (NEW) ---
    print("\n[STEP 3] Syncing to Dropbox...")
    try:
        # Initialize with Refresh Token (Long-term access)
        dbx = dropbox.Dropbox(
            app_key=userdata.get('DBX_APP_KEY'),
            app_secret=userdata.get('DBX_APP_SECRET'),
            oauth2_refresh_token=userdata.get('DBX_REFRESH_TOKEN')
        )

        # Upload the Zip
        target_path = f"/Backups/{zip_name}"
        with open(zip_name, "rb") as f:
            dbx.files_upload(f.read(), target_path, mode=dropbox.files.WriteMode.overwrite)

        print(f"    ✅ Dropbox Success: {target_path}")

    except Exception as e:
        print(f"    ❌ Dropbox Upload Failed: {e}")
        print("       (Check DBX_APP_KEY, DBX_APP_SECRET, DBX_REFRESH_TOKEN in Colab Secrets)")

    # --- 5. LOCAL DOWNLOAD (SAFETY NET) ---
    print("\n[STEP 4] Triggering Local Download...")
    try:
        files.download(zip_name)
    except Exception as e:
        print(f"    ⚠️ Auto-download blocked: {e}")

    print("✅ Archival Process Complete.")

# Run it
run_archiver()

      [CELL 6] MANUAL ARCHIVER + DROPBOX SYNC
📦 Compressing 20 files into Yates_Study_Backup_2026-02-23_2044.zip...
    ✅ Archive Created: Yates_Study_Backup_2026-02-23_2044.zip (2025.7 KB)

[STEP 2] Uploading to Web Server (FTP)...
    ✅ FTP Success: /ons-study/backups/Yates_Study_Backup_2026-02-23_2044.zip

[STEP 3] Syncing to Dropbox...
    ✅ Dropbox Success: /Backups/Yates_Study_Backup_2026-02-23_2044.zip

[STEP 4] Triggering Local Download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Archival Process Complete.


In [ ]:
# @title [CELL Manual Zip & Download]
import os
import zipfile
import pytz
from datetime import datetime
from google.colab import files

print("="*60)
print("      [CELL 7] MANUAL ZIP & DOWNLOADER")
print("="*60)

# Create a timestamped zip file name
est = pytz.timezone('US/Eastern')
timestamp = datetime.now(est).strftime("%Y-%m-%d_%H%M")
zip_filename = f"Yates_Study_Manual_Upload_{timestamp}.zip"

# Find all the files we normally FTP
extensions = ('.html', '.shtml', '.htm', '.csv')
files_to_pack = [f for f in os.listdir('.') if f.lower().endswith(extensions) and "sample_data" not in f]

if not files_to_pack:
    print("❌ No files found to zip! Make sure you ran the Builder cells first.")
else:
    print(f"📦 Found {len(files_to_pack)} files. Compressing into {zip_filename}...\n")

    # Create the zip archive
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zf:
        for file in files_to_pack:
            zf.write(file)
            print(f"  + Added: {file}")

    print(f"\n✅ Zip file created successfully! ({os.path.getsize(zip_filename)/1024:.1f} KB)")

    # Trigger the browser download
    print("⬇️ Prompting browser to download...")
    try:
        files.download(zip_filename)
        print("🎉 Download initiated! You can now manually upload these via FileZilla/Cyberduck.")
    except Exception as e:
        print(f"❌ Auto-download blocked by browser: {e}")
        print(f"👉 You can manually download '{zip_filename}' by clicking the Folder icon 📁 on the far left menu.")

      [CELL 7] MANUAL ZIP & DOWNLOADER
📦 Found 21 files. Compressing into Yates_Study_Manual_Upload_2026-02-22_0937.zip...

  + Added: contents.shtml
  + Added: subscribe.shtml
  + Added: match_to_unmasked.csv
  + Added: ons_yates_dna_register.shtml
  + Added: research_admin.html
  + Added: brick_wall_buster.shtml
  + Added: ons_yates_dna_register_participants.shtml
  + Added: dna_dossier.html
  + Added: engine_database.csv
  + Added: share_dna.shtml
  + Added: lineage_proof.html
  + Added: admin_singletons_participants.shtml
  + Added: proof_consolidator.html
  + Added: dna_theory_of_the_case.htm
  + Added: just-trees-az.shtml
  + Added: yates_ancestor_register.shtml
  + Added: just-trees.shtml
  + Added: data_glossary.shtml
  + Added: biological_proof.html
  + Added: admin_singletons.shtml
  + Added: dna_network.shtml

✅ Zip file created successfully! (1639.3 KB)
⬇️ Prompting browser to download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🎉 Download initiated! You can now manually upload these via FileZilla/Cyberduck.


In [ ]:
# @title GEDCOM Search: The Gremlin Hunter
import os

def find_errant_participant(search_term="yatesjohnrob"):
    print("="*75)
    print(f"      GEDCOM SEARCH: LOOKING FOR '{search_term}'")
    print("="*75)

    # Find the original GEDCOM file
    ged_files = [f for f in os.listdir('.') if f.lower().endswith('.ged') and "_processed" not in f.lower()]
    if not ged_files:
        return print("❌ No original GEDCOM found.")

    ged_file = sorted(ged_files, key=lambda x: os.path.getmtime(x), reverse=True)[0]
    print(f"🔍 Scanning File: {ged_file}\n")

    current_id = None
    current_name = "Unknown"
    matches_found = 0

    print(f"{'ID#'.ljust(12)} | {'NAME'.ljust(30)} | EXACT LINE FOUND")
    print("-" * 75)

    with open(ged_file, 'r', encoding='utf-8', errors='replace') as f:
        for line_num, line in enumerate(f, 1):
            line_clean = line.strip()
            parts = line_clean.split(" ", 2)
            if len(parts) < 2: continue

            lvl = parts[0]
            tag = parts[1]
            val = parts[2] if len(parts) > 2 else ""

            # Track the current individual block
            if lvl == "0" and "INDI" in val:
                current_id = tag.replace("@", "")
                current_name = "Unknown"

            # Track the Name so we know who we are looking at
            elif lvl == "1" and tag == "NAME":
                current_name = val.replace("/", "").strip()

            # Trigger if the search term is anywhere in this line
            if search_term.lower() in line_clean.lower():
                if current_id:
                    print(f"{current_id.ljust(12)} | {current_name[:28].ljust(30)} | {line_clean}")
                else:
                    print(f"{'N/A'.ljust(12)} | {'(Outside INDI block)'.ljust(30)} | {line_clean}")
                matches_found += 1

    print("-" * 75)
    print(f"✅ Found {matches_found} total mentions of '{search_term}'.")

find_errant_participant("yatesjohnrob")

      GEDCOM SEARCH: LOOKING FOR 'yatesjohnrob'
🔍 Scanning File: yates_study_2025.ged

ID#          | NAME                           | EXACT LINE FOUND
---------------------------------------------------------------------------
I51017       | Terri Ann Yates                | 2 NPFX 361&yatesjohnrob
I51033       | Cynthia Lou Miller             | 2 NPFX 20&yatesjohnrob
I51044       | Rhonda Rowe                    | 2 NPFX 19&yatesjohnrob
---------------------------------------------------------------------------
✅ Found 3 total mentions of 'yatesjohnrob'.
